In [10]:
%pip install schedule

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:


"""

Ce script télécharge régulièrement les nouvelles données sismiques depuis l'API IPGP
et les ajoute à une base de données existante.
"""

import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import schedule
import time
import os
import logging
from urllib.parse import urlparse
import io

# Configuration du logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("collecte_seismes_ipgp.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("CollecteSeismesIPGP")

# =============================================================================
# CONFIGURATION - À MODIFIER SELON VOS BESOINS
# =============================================================================

# URL de l'API IPGP
DATA_URL = "https://ws.ipgp.fr/fdsnws/event/1/query?minlatitude=-13.543702&maxlatitude=-12.398181&minlongitude=44.708653&maxlongitude=46.417027&minmagnitude=0&format=text&nodata=404"

# Chemin vers votre fichier de base de données corrigé
DATABASE_FILE = "NewDataseisme_corrige.csv"

# Colonnes de votre base existante
COLONNES_EXISTANTES = ["Date", "Magnitude", "Latitude", "Longitude", "Profondeur", "origine"]

# Valeur par défaut pour la colonne "origine" dans les nouvelles données
ORIGINE_DEFAULT = "5"  # 5 = IPGP (à adapter selon votre nomenclature)

# Fréquence de collecte - par défaut, une fois par jour à 2h du matin
HEURE_COLLECTE = "02:00"

# =============================================================================
# FONCTIONS UTILITAIRES
# =============================================================================

def extract_year(date_str):
    """
    Extrait l'année d'une chaîne de date au format JJ/MM/AAAA ou JJ/MM/AA
    """
    if not isinstance(date_str, str):
        return None
    
    try:
        # Si format ISO
        if 'T' in date_str and '-' in date_str:
            return int(date_str.split('-')[0])
        
        # Format standard JJ/MM/AAAA
        parts = date_str.split('/')
        if len(parts) < 3:
            return None
        
        year_part = parts[2].split(' ')[0]
        # Si année à 2 chiffres, ajouter "20" devant
        if len(year_part) == 2:
            year = int("20" + year_part)
        else:
            year = int(year_part)
        return year
    except Exception as e:
        logger.debug(f"Erreur lors de l'extraction de l'année de '{date_str}': {e}")
        return None

def normalize_date_format(date_str):
    """
    Normalise le format de date vers JJ/MM/AAAA HH:MM
    """
    if not isinstance(date_str, str):
        return date_str
    
    try:
        # Si la date est au format ISO
        if 'T' in date_str and '-' in date_str:
            # Gérer les cas avec ou sans secondes décimales et avec ou sans Z
            date_str_cleaned = date_str.replace('Z', '')
            if '.' in date_str_cleaned:
                # Supprimer les millisecondes
                date_parts = date_str_cleaned.split('.')
                date_str_cleaned = date_parts[0]
            
            # Utiliser dateutil.parser pour une analyse plus robuste
            try:
                from dateutil import parser
                dt = parser.parse(date_str_cleaned)
            except ImportError:
                # Fallback si dateutil n'est pas disponible
                dt = datetime.fromisoformat(date_str_cleaned.replace('Z', '+00:00'))
            
            return dt.strftime('%d/%m/%Y %H:%M')
        
        # Format JJ/MM/AA(AA) HH:MM
        parts = date_str.split('/')
        if len(parts) < 3:
            return date_str
        
        day = parts[0].strip()
        month = parts[1].strip()
        
        # Traiter l'année et l'heure
        year_time = parts[2].split(' ')
        year = year_time[0].strip()
        time = year_time[1].strip() if len(year_time) > 1 else "00:00"
        
        # Si l'année est à 2 chiffres, ajouter "20" devant
        if len(year) == 2:
            year = "20" + year
        
        # Reconstruire la date au format standard
        return f"{day.zfill(2)}/{month.zfill(2)}/{year} {time}"
    except Exception as e:
        logger.debug(f"Erreur lors de la normalisation de '{date_str}': {e}")
        return date_str

def parse_datetime(date_str):
    """
    Convertit différents formats de date en objet datetime
    Gère à la fois les formats ISO et JJ/MM/AAAA
    """
    if not isinstance(date_str, str):
        return None
    
    try:
        # Si c'est déjà un datetime
        if isinstance(date_str, datetime):
            return date_str
        
        # Si format ISO
        if 'T' in date_str and '-' in date_str:
            date_str_cleaned = date_str.replace('Z', '')
            if '.' in date_str_cleaned:
                # Supprimer les millisecondes si nécessaire
                date_parts = date_str_cleaned.split('.')
                date_str_cleaned = date_parts[0]
            
            try:
                from dateutil import parser
                return parser.parse(date_str_cleaned)
            except ImportError:
                return datetime.fromisoformat(date_str_cleaned)
        
        # Format JJ/MM/AAAA HH:MM
        formats_to_try = [
            '%d/%m/%Y %H:%M',
            '%d/%m/%y %H:%M',
            '%d/%m/%Y',
            '%d/%m/%y'
        ]
        
        for fmt in formats_to_try:
            try:
                return datetime.strptime(date_str, fmt)
            except ValueError:
                continue
        
        # Si tous échouent, essayer un parser plus flexible
        try:
            from dateutil import parser
            return parser.parse(date_str, dayfirst=True)
        except (ImportError, ValueError):
            pass
        
        logger.warning(f"Impossible de parser la date: {date_str}")
        return None
    
    except Exception as e:
        logger.error(f"Erreur lors du parsing de la date '{date_str}': {e}")
        return None

def create_unique_id(row):
    """
    Crée un identifiant unique pour chaque entrée basé sur ses attributs
    """
    try:
        # Formater les nombres avec une précision fixe
        lat = f"{float(str(row['Latitude']).replace(',', '.')):.6f}"
        lon = f"{float(str(row['Longitude']).replace(',', '.')):.6f}"
        mag = f"{float(str(row['Magnitude']).replace(',', '.')):.2f}"
        
        # Construire l'identifiant unique
        return f"{row['Date']}_{lat}_{lon}_{mag}"
    except Exception as e:
        # En cas d'erreur, utiliser les valeurs brutes
        return f"{row['Date']}_{row['Latitude']}_{row['Longitude']}_{row['Magnitude']}"

def parse_ipgp_data(text_data):
    """
    Parse les données au format texte de l'API IPGP
    Format attendu: format texte avec un entête commençant par #
    Exemple:
    #EventID | Time | Latitude | Longitude | Depth/km | Author | Catalog | Contributor | ContributorID | MagType | Magnitude | MagAuthor | EventLocationName
    IPGP...  | 2025-05-01T14:30:00.0Z | -12.8123 | 45.3456 | 10.5 | ...
    """
    logger.info("Analyse des données IPGP")
    
    try:
        # Extraire les lignes non commentées
        data_lines = [line for line in text_data.strip().split('\n') if not line.startswith('#')]
        
        # S'il n'y a pas de données, retourner un DataFrame vide
        if not data_lines:
            logger.warning("Aucune donnée trouvée dans la réponse IPGP")
            return pd.DataFrame(columns=COLONNES_EXISTANTES)
        
        # Obtenir les en-têtes (dernière ligne commentée)
        header_lines = [line for line in text_data.strip().split('\n') if line.startswith('#')]
        if not header_lines:
            logger.warning("Aucun en-tête trouvé dans les données IPGP, utilisation d'en-têtes par défaut")
            # En-têtes par défaut selon la documentation FDSN
            headers = ["EventID", "Time", "Latitude", "Longitude", "Depth/km", "Author", "Catalog", 
                      "Contributor", "ContributorID", "MagType", "Magnitude", "MagAuthor", "EventLocationName"]
        else:
            # Utiliser le dernier en-tête trouvé
            headers = [h.strip() for h in header_lines[-1].replace('#', '').split('|')]
        
        # Créer un DataFrame à partir des données
        rows = []
        for line in data_lines:
            values = [v.strip() for v in line.split('|')]
            # S'assurer que nous avons le bon nombre de valeurs
            if len(values) == len(headers):
                rows.append(dict(zip(headers, values)))
            else:
                logger.warning(f"Ligne ignorée car nombre de colonnes incorrect: {line}")
        
        df = pd.DataFrame(rows)
        logger.info(f"DataFrame créé avec {len(df)} lignes et {len(df.columns)} colonnes")
        logger.info(f"Colonnes disponibles: {df.columns.tolist()}")
        
        # Mapper les colonnes IPGP vers notre format
        df_mapped = pd.DataFrame()
        
        # Date (convertir de ISO à notre format)
        if 'Time' in df.columns:
            # Conserver la date originale pour le debugging
            df['Original_Date'] = df['Time']
            logger.info(f"Exemple de date originale: {df['Time'].iloc[0] if len(df) > 0 else 'N/A'}")
            
            # Convertir au format attendu
            df_mapped['Date'] = df['Time'].apply(normalize_date_format)
            logger.info(f"Exemple de date convertie: {df_mapped['Date'].iloc[0] if len(df_mapped) > 0 else 'N/A'}")
        
        # Magnitude
        if 'Magnitude' in df.columns:
            df_mapped['Magnitude'] = df['Magnitude'].astype(str).str.replace('.', ',')
        
        # Latitude
        if 'Latitude' in df.columns:
            df_mapped['Latitude'] = df['Latitude'].astype(str).str.replace('.', ',')
        
        # Longitude
        if 'Longitude' in df.columns:
            df_mapped['Longitude'] = df['Longitude'].astype(str).str.replace('.', ',')
        
        # Profondeur
        if 'Depth/km' in df.columns:
            df_mapped['Profondeur'] = df['Depth/km'].astype(str).str.replace('.', ',')
        
        # Origine (valeur par défaut pour IPGP)
        df_mapped['origine'] = ORIGINE_DEFAULT
        
        # Vérifier que toutes les colonnes nécessaires sont présentes
        missing_cols = set(COLONNES_EXISTANTES) - set(df_mapped.columns)
        if missing_cols:
            logger.warning(f"Colonnes manquantes après le mapping: {missing_cols}")
            for col in missing_cols:
                df_mapped[col] = np.nan
        
        logger.info(f"Données IPGP transformées: {len(df_mapped)} lignes")
        return df_mapped[COLONNES_EXISTANTES]
    
    except Exception as e:
        logger.error(f"Erreur lors de l'analyse des données IPGP: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return pd.DataFrame(columns=COLONNES_EXISTANTES)

# =============================================================================
# FONCTION PRINCIPALE DE COLLECTE
# =============================================================================

def collecter_nouvelles_donnees():
    """
    Fonction principale qui collecte les nouvelles données et les ajoute à la base existante
    Ne collecte que les données plus récentes que la dernière date dans la base existante
    """
    logger.info("Début de la collecte des nouvelles données sismiques")
    
    try:
        # 0. Vérifier si la base de données existante existe et obtenir la date la plus récente
        date_derniere_entree = None
        date_derniere_dt = None
        
        if os.path.exists(DATABASE_FILE):
            try:
                df_existing = pd.read_csv(DATABASE_FILE, sep=';', decimal=',')
                logger.info(f"Base de données existante chargée: {len(df_existing)} lignes")
                
                # Afficher quelques exemples de dates pour le debugging
                if 'Date' in df_existing.columns and len(df_existing) > 0:
                    logger.info(f"Exemples de dates dans la base existante: {df_existing['Date'].iloc[:3].tolist()}")
                    
                    # Trouver la date la plus récente dans la base de données
                    # Utiliser notre fonction robuste pour convertir les dates
                    df_existing['Date_dt'] = df_existing['Date'].apply(parse_datetime)
                    df_existing = df_existing.sort_values('Date_dt', ascending=False, na_position='last')
                    
                    # Ne garder que les lignes avec des dates valides
                    df_existing = df_existing[df_existing['Date_dt'].notna()]
                    
                    if len(df_existing) > 0:
                        date_derniere_entree = df_existing.iloc[0]['Date']
                        date_derniere_dt = df_existing.iloc[0]['Date_dt']
                        
                        logger.info(f"Date la plus récente dans la base: {date_derniere_entree} (parsed as {date_derniere_dt})")
                    else:
                        logger.warning("Aucune date valide trouvée dans la base existante")
                    
                    # Supprimer la colonne temporaire
                    df_existing = df_existing.drop('Date_dt', axis=1)
            except Exception as e:
                logger.error(f"Erreur lors de la lecture de la base existante: {e}")
                import traceback
                logger.error(traceback.format_exc())
                date_derniere_entree = None
        else:
            logger.warning(f"La base de données {DATABASE_FILE} n'existe pas encore")
        
        # 1. Télécharger les nouvelles données depuis l'API IPGP
        logger.info(f"Téléchargement des données depuis {DATA_URL}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(DATA_URL, headers=headers, timeout=60)
        if response.status_code != 200:
            logger.error(f"Erreur lors du téléchargement des données: Code {response.status_code}")
            if response.status_code == 404 and response.text.strip() == "404 Not Found: No data available":
                logger.info("Réponse normale de l'API: aucune donnée disponible pour les critères spécifiés")
            else:
                logger.error(f"Contenu de la réponse: {response.text[:500]}...")
            return
        
        # Afficher un échantillon de la réponse pour le debugging
        logger.info(f"Échantillon des données reçues: {response.text[:500]}...")
        
        # 2. Analyser les données IPGP
        df_new = parse_ipgp_data(response.text)
        logger.info(f"Données téléchargées et analysées: {len(df_new)} lignes")
        
        if len(df_new) == 0:
            logger.info("Aucune nouvelle donnée disponible. Fin de la collecte.")
            return
        
        # 3. Si la base de données n'existe pas, créer une nouvelle base avec toutes les données
        if not os.path.exists(DATABASE_FILE) or not date_derniere_dt:
            if not os.path.exists(DATABASE_FILE):
                logger.warning(f"La base de données {DATABASE_FILE} n'existe pas. Création d'une nouvelle base...")
            else:
                logger.warning("Aucune date valide trouvée dans la base existante. Création d'une nouvelle base...")
            
            df_new.to_csv(DATABASE_FILE, sep=';', index=False)
            logger.info(f"Base de données créée avec {len(df_new)} entrées")
            return
        
        # 4. Filtrer pour ne garder que les données plus récentes que la dernière date connue
        # Convertir les dates du nouveau fichier avec notre fonction robuste
        df_new['Date_dt'] = df_new['Date'].apply(parse_datetime)
        
        # Afficher des exemples pour le debugging
        if len(df_new) > 0:
            logger.info(f"Exemples de dates dans les nouvelles données: {df_new['Date'].iloc[:3].tolist()}")
            logger.info(f"Dates converties: {[str(dt) for dt in df_new['Date_dt'].iloc[:3].tolist()]}")
        
        # Filtrer les entrées plus récentes que la dernière date connue
        df_new_recent = df_new[df_new['Date_dt'] > date_derniere_dt].copy()
        
        logger.info(f"Données filtrées par date: {len(df_new_recent)} entrées plus récentes que {date_derniere_entree}")
        
        # Si aucune donnée plus récente, terminer
        if len(df_new_recent) == 0:
            logger.info("Aucune nouvelle donnée plus récente trouvée. Fin de la collecte.")
            return
        
        # Supprimer la colonne temporaire
        df_new_recent = df_new_recent.drop('Date_dt', axis=1)
        
        # 5. Ajouter les nouvelles données à la base existante
        df_combined = pd.concat([df_existing, df_new_recent], ignore_index=True)
        
        # 6. Trier par date
        logger.info("Tri des données par date...")
        df_combined['Date_dt'] = df_combined['Date'].apply(parse_datetime)
        df_combined = df_combined.sort_values('Date_dt', ascending=False, na_position='last')
        df_combined = df_combined.drop('Date_dt', axis=1)
        
        # 7. Sauvegarder la base mise à jour
        # Créer une sauvegarde de la base existante avant de la remplacer
        backup_file = f"{DATABASE_FILE}.bak"
        df_existing.to_csv(backup_file, sep=';', index=False)
        logger.info(f"Sauvegarde créée: {backup_file}")
        
        # Sauvegarder la nouvelle version
        df_combined.to_csv(DATABASE_FILE, sep=';', index=False)
        logger.info(f"Base de données mise à jour avec {len(df_new_recent)} nouvelles entrées")
        
        # Afficher quelques statistiques
        if 'Date' in df_combined.columns:
            df_combined['Année'] = df_combined['Date'].apply(extract_year)
            year_counts = df_combined['Année'].value_counts().sort_index()
            logger.info(f"Répartition des années dans la base mise à jour: \n{year_counts}")
            df_combined = df_combined.drop('Année', axis=1)
    
    except Exception as e:
        logger.error(f"Erreur lors de la collecte des données: {e}")
        import traceback
        logger.error(traceback.format_exc())

# =============================================================================
# PROGRAMME PRINCIPAL
# =============================================================================

def main():
    """
    Fonction principale du programme
    """
    logger.info("=" * 80)
    logger.info(f"DÉMARRAGE DU PROGRAMME DE COLLECTE DE DONNÉES SISMIQUES IPGP - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    logger.info(f"Base de données: {DATABASE_FILE}")
    logger.info(f"Source des données: {DATA_URL}")
    logger.info(f"Colonnes désirées: {COLONNES_EXISTANTES}")
    logger.info(f"Heure de collecte quotidienne: {HEURE_COLLECTE}")
    logger.info("=" * 80)
    
    # Collecter les données immédiatement au lancement
    collecter_nouvelles_donnees()
    
    # Programmer une collecte quotidienne
    schedule.every().day.at(HEURE_COLLECTE).do(collecter_nouvelles_donnees)
    logger.info(f"Collecte programmée tous les jours à {HEURE_COLLECTE}")
    
    # Boucle principale pour exécuter les tâches programmées
    try:
        while True:
            schedule.run_pending()
            time.sleep(60)  # Vérifier toutes les minutes
    except KeyboardInterrupt:
        logger.info("\nProgramme arrêté par l'utilisateur")
    except Exception as e:
        logger.error(f"Erreur dans la boucle principale: {e}")
        import traceback
        logger.error(traceback.format_exc())

# Point d'entrée du script
if __name__ == "__main__":
    main()

2025-05-09 18:27:04,451 - INFO - ================================================================================
2025-05-09 18:27:04,491 - INFO - DÉMARRAGE DU PROGRAMME DE COLLECTE DE DONNÉES SISMIQUES IPGP - 2025-05-09 18:27:04
2025-05-09 18:27:04,503 - INFO - Base de données: NewDataseisme_corrige.csv
2025-05-09 18:27:04,507 - INFO - Source des données: https://ws.ipgp.fr/fdsnws/event/1/query?minlatitude=-13.543702&maxlatitude=-12.398181&minlongitude=44.708653&maxlongitude=46.417027&minmagnitude=0&format=text&nodata=404
2025-05-09 18:27:04,516 - INFO - Colonnes désirées: ['Date', 'Magnitude', 'Latitude', 'Longitude', 'Profondeur', 'origine']
2025-05-09 18:27:04,523 - INFO - Heure de collecte quotidienne: 02:00
2025-05-09 18:27:04,529 - INFO - ================================================================================
2025-05-09 18:27:04,538 - INFO - Début de la collecte des nouvelles données sismiques
2025-05-09 18:27:05,578 - INFO - Base de données existante chargée: 14216 lig

In [1]:
######### Tableau de bord des séismes avec esthétique améliorée #########

%pip install pandas matplotlib ipywidgets python-dateutil seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime
from dateutil import parser

# Configuration pour un style plus esthétique
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("notebook", font_scale=1.2)
sns.set_palette("deep")

# Couleurs personnalisées
COLORS = {
    'primary': '#3498db',   # Bleu
    'secondary': '#e74c3c',  # Rouge
    'accent': '#2ecc71',    # Vert
    'neutral': '#95a5a6',   # Gris
    'dark': '#2c3e50',      # Bleu foncé
    'light': '#ecf0f1'      # Blanc cassé
}

# CSS pour les widgets et le tableau de bord
CSS = """
<style>
.widget-label {
    font-size: 1.1em;
    font-weight: bold;
    color: #2c3e50;
}
.section-title {
    background-color: #3498db;
    color: white;
    padding: 10px 15px;
    border-radius: 5px;
    margin-top: 20px;
    margin-bottom: 15px;
    font-weight: bold;
}
.dashboard-title {
    background-color: #2c3e50;
    color: white;
    padding: 15px;
    border-radius: 8px;
    margin-bottom: 25px;
    text-align: center;
    font-size: 1.4em;
}
.results-container {
    background-color: #ecf0f1;
    padding: 15px;
    border-radius: 8px;
    margin-top: 10px;
    margin-bottom: 20px;
    border-left: 5px solid #3498db;
}
.footer {
    font-size: 0.8em;
    color: #7f8c8d;
    text-align: center;
    margin-top: 30px;
    font-style: italic;
}
</style>
"""

# Configuration pour afficher plus de colonnes/lignes si nécessaire
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Fonction pour traiter les formats de date multiples
def parse_date_flexible(date_str):
    """
    Fonction robuste pour parser différents formats de date
    """
    if pd.isna(date_str):
        return None
        
    try:
        # Si c'est déjà un datetime
        if isinstance(date_str, datetime):
            return date_str
            
        # Nettoyer la chaîne si nécessaire
        date_str = str(date_str).strip()
        
        # Essayer avec plusieurs formats
        formats_to_try = [
            '%d/%m/%Y %H:%M:%S',
            '%d/%m/%Y %H:%M',
            '%Y-%m-%dT%H:%M:%S.%f',  # Format ISO avec millisecondes
            '%Y-%m-%dT%H:%M:%S',     # Format ISO sans millisecondes
            '%d/%m/%y %H:%M',
            '%Y-%m-%d %H:%M:%S'
        ]
        
        for fmt in formats_to_try:
            try:
                return datetime.strptime(date_str, fmt)
            except:
                continue
                
        # Essayer d'utiliser dateutil.parser en dernier recours
        try:
            # Pour les formats JJ/MM/YYYY, indiquer dayfirst=True
            if '/' in date_str and len(date_str.split('/')[0]) <= 2:
                return parser.parse(date_str, dayfirst=True)
            else:
                return parser.parse(date_str)
        except:
            print(f"Format de date non reconnu: {date_str}")
            return None
    except Exception as e:
        print(f"Erreur lors du parsing de la date '{date_str}': {e}")
        return None

# Charger les données
def charger_donnees():
    # Essayer d'abord avec le séparateur point-virgule
    try:
        df = pd.read_csv('NewDataseisme_corrige.csv', sep=';')
    except:
        # Si ça échoue, essayer avec la virgule
        try:
            df = pd.read_csv('NewDataseisme_corrige.csv', sep=',')
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier: {e}")
            return None
    
    # Afficher un aperçu
    print("Aperçu des données:")
    display(df.head())
    print(f"Nombre total de lignes: {len(df)}")
    
    # Standardiser les noms de colonnes (normaliser la casse)
    df.columns = [col.lower() for col in df.columns]
    
    # Trouver la colonne de date
    date_column = None
    for col in df.columns:
        if 'date' in col.lower() or 'time' in col.lower() or 'heure' in col.lower():
            date_column = col
            break
    
    if not date_column:
        print("Aucune colonne de date trouvée!")
        return None
    
    print(f"Colonne de date identifiée: {date_column}")
    
    # Convertir la colonne de date
    try:
        # Créer une nouvelle colonne datetime
        df['datetime'] = df[date_column].apply(parse_date_flexible)
        
        # Vérifier si la conversion a réussi
        successful_conversions = df['datetime'].notna().sum()
        failed_conversions = df['datetime'].isna().sum()
        
        print(f"\nConversion des dates: {successful_conversions} succès, {failed_conversions} échecs")
        
        if failed_conversions > 0:
            print(f"Suppression de {failed_conversions} lignes avec des dates invalides")
            df = df.dropna(subset=['datetime'])
        
        # Utiliser la nouvelle colonne datetime pour l'analyse
        date_column = 'datetime'
        
    except Exception as e:
        print(f"Erreur lors de la conversion des dates: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    # Extraire les composants temporels
    df['annee'] = df[date_column].dt.year
    df['mois'] = df[date_column].dt.month
    df['jour'] = df[date_column].dt.day
    
    print("\nVérification des années disponibles:")
    print(sorted(df['annee'].unique()))
    
    return df, date_column

# Charger les données
result = charger_donnees()
if result is None:
    raise ValueError("Impossible de continuer sans données valides!")
else:
    df, date_column = result

# Créer des widgets pour les filtres temporels
annees_disponibles = sorted(df['annee'].unique())

# Widget pour sélectionner l'année
annee_dropdown = widgets.Dropdown(
    options=annees_disponibles,
    value=annees_disponibles[-1] if annees_disponibles else None,  # Sélectionner l'année la plus récente par défaut
    description='Année:',
    layout=widgets.Layout(width='200px'),
)

# Widget pour sélectionner le mois
mois_noms = ["Tous", "Janvier", "Février", "Mars", "Avril", "Mai", "Juin", 
            "Juillet", "Août", "Septembre", "Octobre", "Novembre", "Décembre"]
mois_options = [(mois, i) for i, mois in enumerate(mois_noms)]
mois_dropdown = widgets.Dropdown(
    options=mois_options,
    value=0,  # 0 = tous les mois
    description='Mois:',
    layout=widgets.Layout(width='200px'),
)

# Zone d'affichage des résultats par période
output_periode = widgets.Output()

# Zone d'affichage du graphique annuel
output_graphique_annuel = widgets.Output()

# Fonction pour mettre à jour les résultats
def update_resultats(change):
    with output_periode:
        clear_output()
        
        # Récupérer les valeurs des filtres
        annee_selectionnee = annee_dropdown.value
        mois_selectionne = mois_dropdown.value
        
        # Créer les masques de filtrage
        mask_annee = df['annee'] == annee_selectionnee
        
        # Appliquer le filtre d'année
        df_filtered = df[mask_annee].copy()
        
        # Appliquer le filtre de mois si nécessaire
        if mois_selectionne > 0:
            df_filtered = df_filtered[df_filtered['mois'] == mois_selectionne]
        
        # Afficher les résultats
        nb_seismes = len(df_filtered)
        
        if nb_seismes == 0:
            display(HTML("<div style='color:#e74c3c; font-weight:bold; padding:10px; background-color:#fdecea; border-radius:5px;'>Aucun séisme ne correspond aux critères de filtrage.</div>"))
            return
        
        # Afficher le nombre de séismes et la période
        info_html = f"""
        <div style='background-color:#eafaf1; padding:15px; border-radius:8px; margin-bottom:20px; border-left:5px solid #2ecc71;'>
            <span style='font-size:1.3em; font-weight:bold;'>{nb_seismes} séismes</span> correspondent aux critères sélectionnés<br>
            <span style='color:#2c3e50;'>Période: du {df_filtered[date_column].min().strftime('%d/%m/%Y')} au {df_filtered[date_column].max().strftime('%d/%m/%Y')}</span>
        </div>
        """
        display(HTML(info_html))
        
        # Créer une visualisation du nombre de séismes
        plt.figure(figsize=(10, 6))
        
        # Si on a filtré par année et que tous les mois sont sélectionnés
        if mois_selectionne == 0:
            # Agréger par mois
            monthly_counts = df_filtered.groupby('mois').size()
            
            # Créer un index correspondant à tous les mois
            all_months = pd.Series(range(1, 13))
            monthly_counts = monthly_counts.reindex(all_months).fillna(0)
            
            # Noms des mois pour les étiquettes
            month_names = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Juin', 'Juil', 'Août', 'Sep', 'Oct', 'Nov', 'Déc']
            
            # Créer le graphique
            bars = plt.bar(range(1, 13), monthly_counts.values, color=COLORS['primary'], alpha=0.8)
            
            # Ajouter des annotations sur les barres
            for i, bar in enumerate(bars):
                height = bar.get_height()
                if height > 0:
                    plt.text(
                        bar.get_x() + bar.get_width()/2.,
                        height + max(monthly_counts.values) * 0.02,
                        f'{int(height)}',
                        ha='center', va='bottom', fontsize=10
                    )
            
            plt.title(f"Nombre de séismes par mois en {annee_selectionnee}", fontsize=14, pad=20)
            plt.xlabel("Mois", fontsize=12)
            plt.ylabel("Nombre de séismes", fontsize=12)
            plt.xticks(range(1, 13), month_names)
            plt.grid(True, alpha=0.3)
            plt.ylim(0, max(monthly_counts.values) * 1.15)  # Laisser de l'espace pour les annotations
        
        # Si on a filtré par année et mois
        else:
            # Agréger par jour du mois
            daily_counts = df_filtered.groupby('jour').size()
            
            # Nombre de jours dans ce mois
            month_lengths = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
            if annee_selectionnee % 4 == 0 and (annee_selectionnee % 100 != 0 or annee_selectionnee % 400 == 0):
                month_lengths[2] = 29  # Février en année bissextile
            
            days_in_month = month_lengths[mois_selectionne]
            
            # Créer un index pour tous les jours du mois
            all_days = pd.Series(range(1, days_in_month + 1))
            daily_counts = daily_counts.reindex(all_days).fillna(0)
            
            # Créer le graphique
            bars = plt.bar(range(1, days_in_month + 1), daily_counts.values, color=COLORS['secondary'], alpha=0.8)
            
            # Ajouter des annotations sur les barres
            for i, bar in enumerate(bars):
                height = bar.get_height()
                if height > 0:
                    plt.text(
                        bar.get_x() + bar.get_width()/2.,
                        height + max(daily_counts.values) * 0.02 if max(daily_counts.values) > 0 else 0.5,
                        f'{int(height)}',
                        ha='center', va='bottom', fontsize=10
                    )
            
            plt.title(f"Nombre de séismes par jour en {mois_noms[mois_selectionne]} {annee_selectionnee}", fontsize=14, pad=20)
            plt.xlabel("Jour du mois", fontsize=12)
            plt.ylabel("Nombre de séismes", fontsize=12)
            plt.xticks(range(1, days_in_month + 1, 2))
            plt.grid(True, alpha=0.3)
            plt.ylim(0, max(daily_counts.values) * 1.15 if max(daily_counts.values) > 0 else 1)  # Laisser de l'espace pour les annotations
        
        plt.tight_layout()
        
        # Ajouter une bordure et un fond au graphique
        plt.gca().spines['top'].set_visible(True)
        plt.gca().spines['right'].set_visible(True)
        plt.gca().spines['bottom'].set_visible(True)
        plt.gca().spines['left'].set_visible(True)
        
        plt.gca().set_facecolor('#f8f9fa')
        
        plt.show()

# Fonction pour afficher le graphique du nombre total de séismes par année
def afficher_graphique_annuel():
    with output_graphique_annuel:
        clear_output()
        
        # Calculer le nombre de séismes par année
        annual_counts = df.groupby('annee').size()
        
        # Créer le graphique
        plt.figure(figsize=(12, 6))
        
        # Créer un dégradé de couleurs
        colors = [COLORS['dark']] * len(annual_counts)
        
        # Créer les barres
        bars = plt.bar(annual_counts.index, annual_counts.values, color=colors, alpha=0.8)
        
        # Ajouter des annotations sur les barres
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(
                bar.get_x() + bar.get_width()/2.,
                height + max(annual_counts.values) * 0.02,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=11, fontweight='bold'
            )
        
        plt.title("Nombre total de séismes par année", fontsize=16, pad=20)
        plt.xlabel("Année", fontsize=14)
        plt.ylabel("Nombre de séismes", fontsize=14)
        plt.xticks(annual_counts.index, fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.ylim(0, max(annual_counts.values) * 1.15)  # Laisser de l'espace pour les annotations
        
        # Ajouter une ligne de tendance
        z = np.polyfit(range(len(annual_counts.index)), annual_counts.values, 1)
        p = np.poly1d(z)
        plt.plot(annual_counts.index, p(range(len(annual_counts.index))), 
                 linestyle='--', color=COLORS['accent'], linewidth=2, 
                 label=f"Tendance: {z[0]:.1f} séismes/an")
        
        plt.legend(loc='upper left')
        plt.tight_layout()
        
        # Ajouter une bordure et un fond au graphique
        plt.gca().spines['top'].set_visible(True)
        plt.gca().spines['right'].set_visible(True)
        plt.gca().spines['bottom'].set_visible(True)
        plt.gca().spines['left'].set_visible(True)
        
        plt.gca().set_facecolor('#f8f9fa')
        
        plt.show()
        
        # Afficher un tableau récapitulatif
        summary_html = """
        <div style='background-color:#eef5fb; padding:15px; border-radius:8px; margin-top:20px; border-left:5px solid #3498db;'>
            <h3 style='margin-top:0; color:#2c3e50;'>Récapitulatif par année</h3>
            <table style='width:100%; border-collapse:collapse;'>
                <tr style='background-color:#3498db; color:white;'>
                    <th style='padding:8px; text-align:left; border:1px solid #ddd;'>Année</th>
                    <th style='padding:8px; text-align:right; border:1px solid #ddd;'>Nombre de séismes</th>
                    <th style='padding:8px; text-align:right; border:1px solid #ddd;'>Pourcentage</th>
                </tr>
        """
        
        total_seismes = annual_counts.sum()
        
        for year, count in annual_counts.items():
            percentage = count / total_seismes * 100
            row_color = '#f8f9fa' if year % 2 == 0 else 'white'
            summary_html += f"""
                <tr style='background-color:{row_color};'>
                    <td style='padding:8px; text-align:left; border:1px solid #ddd;'>{year}</td>
                    <td style='padding:8px; text-align:right; border:1px solid #ddd;'>{count}</td>
                    <td style='padding:8px; text-align:right; border:1px solid #ddd;'>{percentage:.1f}%</td>
                </tr>
            """
        
        summary_html += f"""
                <tr style='background-color:#eaf2f8; font-weight:bold;'>
                    <td style='padding:8px; text-align:left; border:1px solid #ddd;'>Total</td>
                    <td style='padding:8px; text-align:right; border:1px solid #ddd;'>{total_seismes}</td>
                    <td style='padding:8px; text-align:right; border:1px solid #ddd;'>100.0%</td>
                </tr>
            </table>
        </div>
        """
        
        display(HTML(summary_html))

# Observer les changements des widgets
annee_dropdown.observe(update_resultats, names='value')
mois_dropdown.observe(update_resultats, names='value')

# Créer la mise en page du tableau de bord
dashboard_title = widgets.HTML(f"""
    {CSS}
    <div class="dashboard-title">Analyse des séismes | {len(df)} séismes enregistrés de {min(annees_disponibles)} à {max(annees_disponibles)}</div>
""")

filtres_section = widgets.HTML('<div class="section-title">Filtrage par période</div>')
resultats_section = widgets.HTML('<div class="results-container" id="resultats-container"></div>')
graphique_annuel_section = widgets.HTML('<div class="section-title">Nombre total de séismes par année</div>')
footer = widgets.HTML('<div class="footer">Tableau de bord créé pour l\'analyse des données sismiques</div>')

tableau_bord = widgets.VBox([
    dashboard_title,
    filtres_section,
    widgets.HBox([annee_dropdown, mois_dropdown], layout=widgets.Layout(justify_content='center')),
    output_periode,
    graphique_annuel_section,
    output_graphique_annuel,
    footer
])

# Afficher le tableau de bord
display(tableau_bord)

# Déclencher l'affichage initial des résultats filtrés
update_resultats(None)

# Afficher le graphique annuel
afficher_graphique_annuel()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Aperçu des données:


,Date,Magnitude,Latitude,Longitude,Profondeur,origine
0,08/05/2025 20:34,"1,998769442","-12,750200","45,561000","51,54",5
1,08/05/2025 20:27,"2,374112553","-12,788200","45,619300","47,24",5
2,08/05/2025 19:16,"1,667391108","-12,566000","45,175300","38,39",5
3,08/05/2025 01:04,"2,025011775","-12,805300","45,580500","49,82",5
4,07/05/2025 18:46,"1,282582543","-12,805500","45,347200","43,46",5


Nombre total de lignes: 14216
Colonne de date identifiée: date

Conversion des dates: 14216 succès, 0 échecs

Vérification des années disponibles:
[np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024), np.int32(2025)]


In [ ]:

######Analyse Spatio-Temporelle des seismes  #####

%pip install pandas numpy matplotlib seaborn ipywidgets folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import folium
from folium.plugins import HeatMap, MarkerCluster
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from datetime import datetime, timedelta
import matplotlib.ticker as ticker
import matplotlib as mpl



# Configuration pour afficher plus de colonnes/lignes si nécessaire
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# Fonction pour charger et préparer les données
"""
Fonction de conversion de dates optimisée pour le tableau de bord des séismes.
Cette fonction combine les approches des deux tableaux de bord et ajoute
des améliorations pour gérer les formats spécifiques détectés dans vos données.
"""

from datetime import datetime
from dateutil import parser
import pandas as pd




def parse_date_flexible(date_str):
    """
    Fonction robuste pour parser différents formats de date
    Gère spécifiquement les formats identifiés dans vos données comme 
    "08/05/2025 01:04" et "14/2/25 6:08"
    """
    if pd.isna(date_str):
        return None
        
    try:
        # Si c'est déjà un datetime
        if isinstance(date_str, datetime):
            return date_str
            
        # Nettoyer la chaîne si nécessaire
        date_str = str(date_str).strip()
        
        # Formats détectés dans vos données
        formats_to_try = [
            '%d/%m/%Y %H:%M',    # Pour "08/05/2025 01:04" (jour/mois/année sur 4 chiffres)
            '%d/%m/%y %H:%M',    # Pour "08/05/25 01:04" (jour/mois/année sur 2 chiffres)
            '%d/%m/%Y %H:%M:%S',
            '%d/%m/%y %H:%M:%S',
            # Formats sans zéros de remplissage (pour "14/2/25 6:08")
            '%d/%-m/%y %-H:%M',  # Linux/Mac
            '%d/%m/%y %H:%M',    # Windows (essai alternatif)
            # Autres formats possibles
            '%Y-%m-%dT%H:%M:%S.%f',  # Format ISO avec millisecondes
            '%Y-%m-%dT%H:%M:%S',     # Format ISO sans millisecondes
            '%Y-%m-%d %H:%M:%S'
        ]
        
        # Essayer tous les formats explicites d'abord
        for fmt in formats_to_try:
            try:
                # Adaptation pour Windows qui ne supporte pas %-
                if '%-' in fmt and '/' in date_str:
                    # Extraire les parties de la date pour un traitement manuel
                    parts = date_str.split()
                    if len(parts) == 2:  # Format "14/2/25 6:08"
                        date_part = parts[0]
                        time_part = parts[1]
                        
                        # Découper les composants de la date
                        day, month, year = date_part.split('/')
                        hour, minute = time_part.split(':')
                        
                        # Convertir en nombres
                        day = int(day)
                        month = int(month)
                        
                        # Déterminer si l'année est sur 2 ou 4 chiffres
                        if len(year) == 2:
                            # Convertir année sur 2 chiffres (20xx)
                            year = 2000 + int(year)
                        else:
                            year = int(year)
                            
                        hour = int(hour)
                        minute = int(minute)
                        
                        # Créer l'objet datetime
                        return datetime(year, month, day, hour, minute)
                else:
                    return datetime.strptime(date_str, fmt)
            except:
                continue
        
        # Si aucun format explicite ne fonctionne, essayer avec dateutil.parser
        try:
            # Pour les formats JJ/MM/YYYY ou JJ/MM/YY, utiliser dayfirst=True
            if '/' in date_str and len(date_str.split('/')[0]) <= 2:
                # Exemple: "14/2/25 6:08" ou "08/05/2025 01:04"
                return parser.parse(date_str, dayfirst=True)
            else:
                return parser.parse(date_str)
        except:
            # Dernière tentative: extraire manuellement les parties
            try:
                if '/' in date_str and ' ' in date_str:
                    # Format supposé: "jour/mois/année heure:minute"
                    date_part, time_part = date_str.split(' ', 1)
                    day, month, year = date_part.split('/')
                    
                    # Gérer le cas où les heures et minutes sont séparées par ':'
                    if ':' in time_part:
                        hour, minute = time_part.split(':', 1)
                    else:
                        hour, minute = time_part, 0
                    
                    # Convertir en nombres et gérer les années sur 2 chiffres
                    day = int(day)
                    month = int(month)
                    year = int(year)
                    if year < 100:  # Année sur 2 chiffres
                        year = 2000 + year
                    
                    hour = int(hour)
                    minute = int(minute)
                    
                    return datetime(year, month, day, hour, minute)
            except Exception as e:
                print(f"Échec de l'extraction manuelle: {e}")
        
        print(f"Format de date non reconnu: {date_str}")
        return None
    except Exception as e:
        print(f"Erreur lors du parsing de la date '{date_str}': {e}")
        return None

# Fonction complète pour charger et préparer les données avec gestion robuste des dates
def charger_donnees():
    # Charger les données
    try:
        df = pd.read_csv('NewDataseisme_corrige.csv', sep=';')
    except:
        try:
            df = pd.read_csv('NewDataseisme_corrige.csv', sep=',')
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier: {e}")
            return None
    
    # Afficher un aperçu
    print("Aperçu des données:")
    display(df.head())
    
    # Afficher quelques exemples de dates pour diagnostic
    print("\nExemples de format de date dans les données:")
    if 'Date' in df.columns:
        # Prendre quelques échantillons aléatoires pour mieux voir la diversité des formats
        date_samples = df['Date'].sample(min(10, len(df))).tolist()
        for sample in date_samples:
            print(sample)
    
    # Convertir la colonne de date avec notre fonction améliorée
    if 'Date' in df.columns:
        print("\nConversion des dates avec parse_date_flexible...")
        
        # Conserver la date originale
        df['Date_orig'] = df['Date']
        
        # Créer une nouvelle colonne pour les dates converties
        df['Date_dt'] = df['Date'].apply(parse_date_flexible)
        
        # Vérifier le succès de la conversion
        success_count = df['Date_dt'].notna().sum()
        fail_count = df['Date_dt'].isna().sum()
        
        print(f"Conversion des dates: {success_count} succès, {fail_count} échecs")
        
        # Si certaines dates n'ont pas pu être converties, afficher des exemples
        if fail_count > 0:
            print(f"Exemples de dates qui n'ont pas pu être converties:")
            failed_examples = df[df['Date_dt'].isna()]['Date'].head(5).tolist()
            for example in failed_examples:
                print(f"  - '{example}'")
        
        # Corriger le format des nombres (virgule à point)
        for col in ['Magnitude', 'Latitude', 'Longitude', 'Profondeur']:
            if col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = df[col].str.replace(',', '.').astype(float)
        
        # Extraire les composantes temporelles depuis la colonne convertie
        if success_count > 0:
            df['Annee'] = df['Date_dt'].dt.year
            df['Mois'] = df['Date_dt'].dt.month
            df['Jour'] = df['Date_dt'].dt.day
            df['Heure'] = df['Date_dt'].dt.hour
            df['JourSemaine'] = df['Date_dt'].dt.dayofweek
            
            # Créer des périodes trimestrielles et semestrielles
            df['Trimestre'] = df['Date_dt'].dt.quarter
            df['Semestre'] = (df['Mois'] <= 6).astype(int) + 1
            
            # Remplacer la colonne Date par Date_dt pour compatibilité avec le reste du code
            df['Date'] = df['Date_dt']
        else:
            print("ERREUR: Aucune date n'a pu être convertie")
            return None
    else:
        print("ERREUR: Colonne 'Date' non trouvée dans les données")
        return None
    
    # Nettoyer les colonnes (supprimer les caractères \r si présents)
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('\r', '')
    
    print(f"Données chargées: {len(df)} enregistrements")
    return df

# Chargement des données
df = charger_donnees()

# Résumé statistique
print("\nRésumé statistique des variables numériques:")
display(df.describe())

# Vérifier les valeurs manquantes
print("\nValeurs manquantes par colonne:")
display(df.isna().sum())

# Information sur la période couverte
print(f"\nPériode couverte: de {df['Date'].min()} à {df['Date'].max()}")
print(f"Durée: {(df['Date'].max() - df['Date'].min()).days} jours")

# ====================== INTERFACE INTERACTIVE ======================

# Créer les widgets pour les filtres
filter_out = widgets.Output()

# Filtres temporels
annees = sorted(df['Annee'].unique())
mois = list(range(1, 13))
mois_noms = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 
             'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
mois_dict = {i+1: nom for i, nom in enumerate(mois_noms)}

# Filtres magnitude
min_mag = float(df['Magnitude'].min())
max_mag = float(df['Magnitude'].max())

# Filtres profondeur
min_prof = float(df['Profondeur'].min())
max_prof = float(df['Profondeur'].max())

# Création des widgets
annee_slider = widgets.IntRangeSlider(
    value=[annees[0], annees[-1]],
    min=annees[0],
    max=annees[-1],
    step=1,
    description='Années:',
    continuous_update=False,
    layout=widgets.Layout(width='70%')
)

mois_checkbox = widgets.SelectMultiple(
    options=[(mois_dict[m], m) for m in mois],
    value=mois,
    description='Mois:',
    layout=widgets.Layout(width='50%', height='100px')
)

magnitude_slider = widgets.FloatRangeSlider(
    value=[min_mag, max_mag],
    min=min_mag,
    max=max_mag,
    step=0.1,
    description='Magnitude:',
    continuous_update=False,
    layout=widgets.Layout(width='70%')
)

profondeur_slider = widgets.FloatRangeSlider(
    value=[min_prof, max_prof],
    min=min_prof,
    max=max_prof,
    step=5,
    description='Profondeur:',
    continuous_update=False,
    layout=widgets.Layout(width='70%')
)

# Types d'analyse
analyse_type = widgets.RadioButtons(
    options=['Distribution temporelle', 'Carte des séismes', 'Analyse par magnitude', 'Corrélations'],
    description='Type d\'analyse:',
    layout=widgets.Layout(width='50%')
)

filtrer_button = widgets.Button(
    description='Appliquer les filtres',
    button_style='primary',
    tooltip='Cliquez pour appliquer les filtres',
    layout=widgets.Layout(width='200px')
)

# Fonction pour appliquer les filtres
def filtrer_donnees(df, annee_range, mois_selected, magnitude_range, profondeur_range):
    df_filtered = df.copy()
    
    # Filtrer par année
    df_filtered = df_filtered[(df_filtered['Annee'] >= annee_range[0]) & 
                              (df_filtered['Annee'] <= annee_range[1])]
    
    # Filtrer par mois
    df_filtered = df_filtered[df_filtered['Mois'].isin(mois_selected)]
    
    # Filtrer par magnitude
    df_filtered = df_filtered[(df_filtered['Magnitude'] >= magnitude_range[0]) & 
                              (df_filtered['Magnitude'] <= magnitude_range[1])]
    
    # Filtrer par profondeur
    df_filtered = df_filtered[(df_filtered['Profondeur'] >= profondeur_range[0]) & 
                              (df_filtered['Profondeur'] <= profondeur_range[1])]
    
    return df_filtered

# Fonction pour créer une carte des séismes
def creer_carte_seismes(df_filtered):
    """
    Crée une carte des séismes avec un marqueur pour le volcan Fani Maoré,
    les îles de Mayotte, et des cercles de distance.
    
    Paramètres:
    - df_filtered: DataFrame contenant les données sismiques filtrées
    """
    # Coordonnées précises du volcan Fani Maoré
    fanimaoré = {
        'nom': 'Fani Maoré',
        'lat': -12.80,  # 12° 48′ sud
        'lon': 45.467   # 45° 28′ est
    }
    
    # Coordonnées des principales îles de Mayotte
    mayotte_iles = [
        {'nom': 'Grande-Terre', 'lat': -12.7817, 'lon': 45.2269, 'type': 'île principale'},
        {'nom': 'Petite-Terre', 'lat': -12.7892, 'lon': 45.2804, 'type': 'île principale'},
        {'nom': 'Mtsamboro', 'lat': -12.6964, 'lon': 45.0845, 'type': 'îlot'},
        {'nom': 'Mbouzi', 'lat': -12.8121, 'lon': 45.2338, 'type': 'îlot'},
        {'nom': 'Bandrélé', 'lat': -12.9085, 'lon': 45.1932, 'type': 'ville'}
    ]
    
    # Créer une carte centrée entre Mayotte et Fani Maoré
    center_lat = (fanimaoré['lat'] + mayotte_iles[0]['lat']) / 2
    center_lon = (fanimaoré['lon'] + mayotte_iles[0]['lon']) / 2
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8,
                  tiles='CartoDB positron')
    
    # Ajouter un titre à la carte
    title_html = '''
        <div style="position: fixed; 
                    top: 10px; left: 50px; width: 350px; height: 30px; 
                    background-color: rgba(255, 255, 255, 0.8);
                    border-radius: 5px; padding: 10px; z-index: 900;">
            <h4 style="margin: 0; text-align: center; color: #2c3e50;">
                Séismes autour de Mayotte et Fani Maoré ({} événements)
            </h4>
        </div>
    '''.format(len(df_filtered))
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Créer différents groupes de calques pour les contrôles
    heat_layer = folium.FeatureGroup(name="Densité des séismes").add_to(m)
    marker_cluster = MarkerCluster(name="Séismes individuels").add_to(m)
    volcan_group = folium.FeatureGroup(name="Fani Maoré", show=True).add_to(m)
    iles_group = folium.FeatureGroup(name="Îles de Mayotte", show=True).add_to(m)
    distance_group = folium.FeatureGroup(name="Cercles de distance", show=True).add_to(m)
    
    # Préparer les données pour la heatmap
    heat_data = [[row['Latitude'], row['Longitude'], row['Magnitude']] 
                for _, row in df_filtered.iterrows()]
    
    # Améliorer les couleurs de la heatmap
    gradient_dict = {'0.4': '#4575b4', '0.65': '#91bfdb', '0.8': '#fee090', '1.0': '#d73027'}
    
    # Ajouter la heatmap
    HeatMap(heat_data, radius=15, blur=10, gradient=gradient_dict).add_to(heat_layer)
    
    # Normaliser les magnitudes pour la coloration
    norm = Normalize(vmin=df_filtered['Magnitude'].min(), vmax=df_filtered['Magnitude'].max())
    import matplotlib as mpl
    cmap = mpl.colormaps['plasma']  # Méthode mise à jour pour obtenir la palette de couleurs
    
    # Fonction haversine pour calculer les distances
    from math import radians, cos, sin, asin, sqrt
    def haversine(lat1, lon1, lat2, lon2):
        # Convertir degrés en radians
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        # Formule haversine
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a))
        r = 6371  # Rayon de la Terre en km
        return c * r
    
    # Ajouter des marqueurs pour chaque séisme
    sample_size = min(500, len(df_filtered))  # Limiter le nombre de marqueurs pour la performance
    for _, row in df_filtered.sample(sample_size).iterrows():
        color = '#%02x%02x%02x' % tuple(int(255 * x) for x in cmap(norm(row['Magnitude']))[:3])
        
        # Calculer les distances
        distance_volcan = haversine(row['Latitude'], row['Longitude'], 
                                   fanimaoré['lat'], fanimaoré['lon'])
        
        # Calculer la distance par rapport à Grande-Terre (Mayotte)
        distance_mayotte = haversine(row['Latitude'], row['Longitude'], 
                                    mayotte_iles[0]['lat'], mayotte_iles[0]['lon'])
        
        # Créer le texte pour le popup
        popup_html = f"""
        <div style="width: 200px; font-family: Arial; font-size: 12px;">
            <h4 style="margin: 0 0 5px 0; color: #2c3e50;">Séisme</h4>
            <hr style="margin: 2px 0; border-color: #eee;">
            <p><b>Date:</b> {row['Date_dt'].strftime('%d/%m/%Y %H:%M')}</p>
            <p><b>Magnitude:</b> <span style="color:{color}; font-weight:bold;">{row['Magnitude']:.2f}</span></p>
            <p><b>Profondeur:</b> {row['Profondeur']:.2f} km</p>
            <p><b>Coordonnées:</b> {row['Latitude']:.4f}, {row['Longitude']:.4f}</p>
            <hr style="margin: 5px 0;">
            <p><b>Distance à Fani Maoré:</b> {distance_volcan:.1f} km</p>
            <p><b>Distance à Mayotte:</b> {distance_mayotte:.1f} km</p>
        </div>
        """
        
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=max(3, row['Magnitude'] * 1.5),
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(popup_html, max_width=300),
        ).add_to(marker_cluster)
    
    # Ajouter un marqueur pour le volcan Fani Maoré
    folium.Marker(
        location=[fanimaoré['lat'], fanimaoré['lon']],
        popup=f"""
        <div style="width: 200px;">
            <h4 style="margin: 0 0 5px 0; color: #e74c3c;">Volcan Fani Maoré</h4>
            <hr style="margin: 2px 0;">
            <p><b>Coordonnées:</b> 12° 48′ sud, 45° 28′ est</p>
            <p><b>Type:</b> Volcan sous-marin</p>
            <p><b>Statut:</b> Actif</p>
        </div>
        """,
        tooltip="Fani Maoré",
        icon=folium.Icon(color='red', icon='info-sign')  # Modifié pour utiliser une icône par défaut
    ).add_to(volcan_group)
    
    # Ajouter les îles de Mayotte
    for ile in mayotte_iles:
        # Différentes icônes selon le type
        if ile['type'] == 'île principale':
            icon = folium.Icon(color='green', icon='info-sign')  # Icône par défaut
        elif ile['type'] == 'îlot':
            icon = folium.Icon(color='green', icon='circle')
        else:
            icon = folium.Icon(color='blue', icon='info-sign')
        
        folium.Marker(
            location=[ile['lat'], ile['lon']],
            popup=f"""
            <div style="width: 200px;">
                <h4 style="margin: 0 0 5px 0; color: #2ecc71;">{ile['nom']}</h4>
                <hr style="margin: 2px 0;">
                <p><b>Type:</b> {ile['type'].capitalize()}</p>
                <p><b>Coordonnées:</b> {abs(ile['lat']):.4f}° S, {ile['lon']:.4f}° E</p>
                <p><b>Distance à Fani Maoré:</b> {haversine(ile['lat'], ile['lon'], fanimaoré['lat'], fanimaoré['lon']):.1f} km</p>
            </div>
            """,
            tooltip=ile['nom'],
            icon=icon
        ).add_to(iles_group)
    
    # Dessiner le contour approximatif de Grande-Terre de Mayotte
    grande_terre_coords = [
        [-12.7355, 45.0767], [-12.6593, 45.1043], [-12.6593, 45.1456],
        [-12.6824, 45.1894], [-12.7309, 45.2250], [-12.7863, 45.2471],
        [-12.8486, 45.2250], [-12.9109, 45.1784], [-12.9363, 45.1264],
        [-12.9132, 45.0629], [-12.8532, 45.0408], [-12.7863, 45.0408],
        [-12.7355, 45.0767]
    ]
    
    folium.Polygon(
        locations=grande_terre_coords,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.3,
        tooltip="Grande-Terre (Mayotte)"
    ).add_to(iles_group)
    
    # Dessiner le contour approximatif de Petite-Terre
    petite_terre_coords = [
        [-12.7723, 45.2757], [-12.7769, 45.2840], [-12.7909, 45.2881],
        [-12.8038, 45.2798], [-12.8015, 45.2715], [-12.7769, 45.2674],
        [-12.7723, 45.2757]
    ]
    
    folium.Polygon(
        locations=petite_terre_coords,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.3,
        tooltip="Petite-Terre (Mayotte)"
    ).add_to(iles_group)
    
    # Ajouter des cercles de distance autour de Fani Maoré
    distances = [5, 10, 20, 50]
    colors = ['#ff0000', '#ff6600', '#ffcc00', '#ffff00']
    
    for i, distance in enumerate(distances):
        folium.Circle(
            location=[fanimaoré['lat'], fanimaoré['lon']],
            radius=distance * 1000,  # Convertir km en mètres
            color=colors[i],
            fill=True,
            fill_opacity=0.1,
            weight=2,
            popup=f"Rayon de {distance} km autour de Fani Maoré"
        ).add_to(distance_group)
    
    # Créer aussi un cercle de distance autour de Grande-Terre pour référence
    folium.Circle(
        location=[mayotte_iles[0]['lat'], mayotte_iles[0]['lon']],
        radius=20 * 1000,  # 20 km
        color='green',
        fill=True,
        fill_opacity=0.1,
        weight=2,
        popup="Rayon de 20 km autour de Mayotte (Grande-Terre)"
    ).add_to(distance_group)
    
    # Ajouter une légende claire
    legend_html = '''
         <div style="position: fixed; 
                     bottom: 50px; right: 50px; width: 200px; height: auto;
                     background-color: white; border-radius: 5px;
                     box-shadow: 0 0 15px rgba(0,0,0,0.2);
                     padding: 10px; z-index: 900;">
             <p style="margin:0; text-align:center; font-weight:bold;">Magnitude des séismes</p>
             <hr style="margin: 5px 0;">
             <div style="display: flex; justify-content: space-between;">
                 <span>Faible</span>
                 <div style="width: 100px; height: 20px; background: linear-gradient(to right, #440154, #482878, #3e4989, #31688e, #26828e, #1f9e89, #35b779, #6ece58, #b5de2b, #fde725);"></div>
                 <span>Forte</span>
             </div>
             <div style="display: flex; justify-content: space-between; margin-top: 5px;">
                 <span>''' + f"{df_filtered['Magnitude'].min():.1f}" + '''</span>
                 <span>''' + f"{df_filtered['Magnitude'].max():.1f}" + '''</span>
             </div>
             <hr style="margin: 10px 0;">
             <p style="margin:0; text-align:center; font-weight:bold;">Points d'intérêt</p>
             <div style="margin-top: 5px;">
                 <div style="display: flex; align-items: center; margin-bottom: 5px;">
                     <div style="background-color: red; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div>
                     <span>Fani Maoré (volcan)</span>
                 </div>
                 <div style="display: flex; align-items: center; margin-bottom: 5px;">
                     <div style="background-color: green; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div>
                     <span>Îles principales</span>
                 </div>
                 <div style="display: flex; align-items: center;">
                     <div style="background-color: blue; width: 15px; height: 15px; border-radius: 50%; margin-right: 5px;"></div>
                     <span>Villes</span>
                 </div>
             </div>
             <hr style="margin: 10px 0;">
             <p style="margin:0; text-align:center; font-weight:bold;">Distances</p>
             <div style="margin-top: 5px;">
                 <div style="display: flex; align-items: center; margin-bottom: 3px;">
                     <div style="width: 15px; height: 15px; background-color: #ff0000; margin-right: 5px; border-radius: 50%;"></div>
                     <span>5 km (Fani Maoré)</span>
                 </div>
                 <div style="display: flex; align-items: center; margin-bottom: 3px;">
                     <div style="width: 15px; height: 15px; background-color: #ff6600; margin-right: 5px; border-radius: 50%;"></div>
                     <span>10 km (Fani Maoré)</span>
                 </div>
                 <div style="display: flex; align-items: center; margin-bottom: 3px;">
                     <div style="width: 15px; height: 15px; background-color: #ffcc00; margin-right: 5px; border-radius: 50%;"></div>
                     <span>20 km (Fani Maoré)</span>
                 </div>
                 <div style="display: flex; align-items: center; margin-bottom: 3px;">
                     <div style="width: 15px; height: 15px; background-color: #ffff00; margin-right: 5px; border-radius: 50%;"></div>
                     <span>50 km (Fani Maoré)</span>
                 </div>
                 <div style="display: flex; align-items: center;">
                     <div style="width: 15px; height: 15px; background-color: green; margin-right: 5px; border-radius: 50%;"></div>
                     <span>20 km (Mayotte)</span>
                 </div>
             </div>
         </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Ajouter un contrôle de calques
    folium.LayerControl().add_to(m)
    
    return m



# Analyse de distribution temporelle
def analyse_temporelle(df_filtered, periode='Mois'):
    plt.figure(figsize=(14, 8))
    
    if periode == 'Annee':
        counts = df_filtered['Annee'].value_counts().sort_index()
        plt.bar(counts.index, counts.values)
        plt.xlabel('Année')
        plt.title(f'Distribution des séismes par année ({len(df_filtered)} séismes)')
        plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        
    elif periode == 'Mois':
        counts = df_filtered['Mois'].value_counts().sort_index()
        plt.bar(counts.index, counts.values)
        plt.xlabel('Mois')
        plt.xticks(range(1, 13), mois_noms, rotation=45)
        plt.title(f'Distribution des séismes par mois ({len(df_filtered)} séismes)')
        
    elif periode == 'Jour':
        counts = df_filtered['Jour'].value_counts().sort_index()
        plt.bar(counts.index, counts.values)
        plt.xlabel('Jour du mois')
        plt.title(f'Distribution des séismes par jour du mois ({len(df_filtered)} séismes)')
        
    elif periode == 'Heure':
        counts = df_filtered['Heure'].value_counts().sort_index()
        plt.bar(counts.index, counts.values)
        plt.xlabel('Heure de la journée')
        plt.xticks(range(0, 24))
        plt.title(f'Distribution des séismes par heure ({len(df_filtered)} séismes)')
        
    elif periode == 'JourSemaine':
        jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
        counts = df_filtered['JourSemaine'].value_counts().sort_index()
        plt.bar(counts.index, counts.values)
        plt.xlabel('Jour de la semaine')
        plt.xticks(range(7), jours, rotation=45)
        plt.title(f'Distribution des séismes par jour de la semaine ({len(df_filtered)} séismes)')
    
    plt.ylabel('Nombre de séismes')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Afficher les statistiques
    print(f"Distribution temporelle par {periode.lower()}:")
    if periode == 'Mois':
        for idx, count in counts.items():
            print(f"{mois_dict[idx]}: {count} séismes ({count/len(df_filtered)*100:.1f}%)")
    elif periode == 'JourSemaine':
        jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
        for idx, count in counts.items():
            print(f"{jours[idx]}: {count} séismes ({count/len(df_filtered)*100:.1f}%)")
    else:
        for idx, count in counts.items():
            print(f"{idx}: {count} séismes ({count/len(df_filtered)*100:.1f}%)")

# Analyse par magnitude
def analyse_magnitude(df_filtered):
    plt.figure(figsize=(14, 10))
    
    # Subplot 1: Distribution des magnitudes
    plt.subplot(2, 2, 1)
    sns.histplot(df_filtered['Magnitude'], bins=30, kde=True)
    plt.title('Distribution des magnitudes')
    plt.xlabel('Magnitude')
    plt.ylabel('Fréquence')
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Magnitude moyenne par année
    plt.subplot(2, 2, 2)
    mag_by_year = df_filtered.groupby('Annee')['Magnitude'].mean()
    plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
    plt.bar(mag_by_year.index, mag_by_year.values)
    plt.title('Magnitude moyenne par année')
    plt.xlabel('Année')
    plt.ylabel('Magnitude moyenne')
    plt.grid(True, alpha=0.3)
    
    # Subplot 3: Magnitude vs Profondeur
    plt.subplot(2, 2, 3)
    plt.scatter(df_filtered['Profondeur'], df_filtered['Magnitude'], alpha=0.5)
    plt.title('Magnitude vs Profondeur')
    plt.xlabel('Profondeur (km)')
    plt.ylabel('Magnitude')
    plt.grid(True, alpha=0.3)
    
    # Subplot 4: Évolution temporelle des magnitudes
    plt.subplot(2, 2, 4)
    plt.scatter(df_filtered['Date'], df_filtered['Magnitude'], alpha=0.5, s=10)
    plt.title('Évolution temporelle des magnitudes')
    plt.xlabel('Date')
    plt.ylabel('Magnitude')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques sur les magnitudes
    print("Statistiques des magnitudes:")
    print(f"Moyenne: {df_filtered['Magnitude'].mean():.2f}")
    print(f"Médiane: {df_filtered['Magnitude'].median():.2f}")
    print(f"Min: {df_filtered['Magnitude'].min():.2f}")
    print(f"Max: {df_filtered['Magnitude'].max():.2f}")
    print(f"Écart-type: {df_filtered['Magnitude'].std():.2f}")
    
    # Répartition par catégorie de magnitude
    bins = [0, 1, 2, 3, 4, 5, float('inf')]
    labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5+']
    df_filtered['MagnitudeCategorie'] = pd.cut(df_filtered['Magnitude'], bins=bins, labels=labels)
    cat_counts = df_filtered['MagnitudeCategorie'].value_counts().sort_index()
    
    print("\nRépartition par catégorie de magnitude:")
    for category, count in cat_counts.items():
        print(f"Magnitude {category}: {count} séismes ({count/len(df_filtered)*100:.1f}%)")

# Analyse des corrélations
def analyse_correlations(df_filtered):
    # Sélectionner les colonnes numériques pertinentes
    cols_numeriques = ['Magnitude', 'Profondeur', 'Latitude', 'Longitude', 'Annee', 'Mois', 'Jour', 'Heure']
    df_num = df_filtered[cols_numeriques]
    
    # Calculer la matrice de corrélation
    corr_matrix = df_num.corr()
    
    # Afficher la matrice de corrélation
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('Matrice de corrélation')
    plt.tight_layout()
    plt.show()
    
    # Identifier les corrélations les plus fortes (en valeur absolue)
    corr_unstack = corr_matrix.unstack()
    corr_unstack = corr_unstack[corr_unstack < 1.0]  # Supprimer les auto-corrélations (= 1.0)
    corr_abs = corr_unstack.abs()
    corr_sorted = corr_abs.sort_values(ascending=False)
    
    print("Corrélations les plus fortes (en valeur absolue):")
    # Convertir items() en liste et prendre les 10 premiers éléments
    top_correlations = list(corr_sorted.items())[:10]
    for i, ((col1, col2), corr_value) in enumerate(top_correlations):
        print(f"{col1} - {col2}: {corr_matrix.loc[col1, col2]:.3f}")

# Fonction principale pour mettre à jour l'analyse
def update_analysis(b):
    with filter_out:
        clear_output(wait=True)
        
        # Récupérer les valeurs des filtres
        annee_range = annee_slider.value
        mois_selected = mois_checkbox.value
        magnitude_range = magnitude_slider.value
        profondeur_range = profondeur_slider.value
        
        # Appliquer les filtres
        df_filtered = filtrer_donnees(df, annee_range, mois_selected, magnitude_range, profondeur_range)
        
        print(f"Données filtrées: {len(df_filtered)} séismes sur {len(df)} ({len(df_filtered)/len(df)*100:.1f}%)")
        
        if len(df_filtered) == 0:
            print("Aucune donnée ne correspond aux critères de filtrage!")
            return
        
        # Effectuer l'analyse sélectionnée
        analysis_type = analyse_type.value
        
        if analysis_type == 'Distribution temporelle':
            # Créer des onglets pour différentes périodes temporelles
            periode_tabs = widgets.Tab()
            children = []
            
            for periode in ['Annee', 'Mois', 'Jour', 'Heure', 'JourSemaine']:
                output = widgets.Output()
                with output:
                    analyse_temporelle(df_filtered, periode)
                children.append(output)
            
            periode_tabs.children = children
            periode_tabs.set_title(0, 'Années')
            periode_tabs.set_title(1, 'Mois')
            periode_tabs.set_title(2, 'Jours')
            periode_tabs.set_title(3, 'Heures')
            periode_tabs.set_title(4, 'Jours semaine')
            
            display(periode_tabs)
            
        elif analysis_type == 'Carte des séismes':
            try:
                m = creer_carte_seismes(df_filtered)
                display(m)
            except Exception as e:
                print(f"Erreur lors de la création de la carte: {e}")
                print("Note: Pour voir les cartes, assurez-vous d'avoir installé folium (pip install folium)")
                
                # Alternative sans folium - carte simple avec Matplotlib
                plt.figure(figsize=(10, 8))
                plt.scatter(df_filtered['Longitude'], df_filtered['Latitude'], 
                            c=df_filtered['Magnitude'], cmap='plasma', alpha=0.6)
                plt.colorbar(label='Magnitude')
                plt.title('Carte des séismes (Alternative simple)')
                plt.xlabel('Longitude')
                plt.ylabel('Latitude')
                plt.grid(True, alpha=0.3)
                plt.show()
                
        elif analysis_type == 'Analyse par magnitude':
            analyse_magnitude(df_filtered)
            
        elif analysis_type == 'Corrélations':
            analyse_correlations(df_filtered)

# Connecter le bouton à la fonction d'actualisation
filtrer_button.on_click(update_analysis)

# Afficher l'interface
print("\n--- INTERFACE D'ANALYSE SPATIO-TEMPORELLE DES SEISMES ---")
controls = widgets.VBox([
    widgets.HBox([annee_slider]),
    widgets.HBox([mois_checkbox]),
    widgets.HBox([magnitude_slider]),
    widgets.HBox([profondeur_slider]),
    widgets.HBox([analyse_type]),
    widgets.HBox([filtrer_button])
])

display(controls)
display(filter_out)

# Lancer l'analyse avec les filtres par défaut
update_analysis(None)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Aperçu des données:


,Date,Magnitude,Latitude,Longitude,Profondeur,origine
0,08/05/2025 20:34,"1,998769442","-12,750200","45,561000","51,54",5
1,08/05/2025 20:27,"2,374112553","-12,788200","45,619300","47,24",5
2,08/05/2025 19:16,"1,667391108","-12,566000","45,175300","38,39",5
3,08/05/2025 01:04,"2,025011775","-12,805300","45,580500","49,82",5
4,07/05/2025 18:46,"1,282582543","-12,805500","45,347200","43,46",5



Exemples de format de date dans les données:
17/7/19 23:57
2/11/19 0:16
20/7/19 9:48
22/4/20 0:34
6/6/18 14:17
6/12/18 1:17
26/6/21 1:19
16/4/20 20:57
2/9/19 23:42
30/7/22 4:10

Conversion des dates avec parse_date_flexible...
Conversion des dates: 14216 succès, 0 échecs
Données chargées: 14216 enregistrements

Résumé statistique des variables numériques:


,Date,Magnitude,Latitude,Longitude,Profondeur,origine,Date_dt,Annee,Mois,Jour,Heure,JourSemaine,Trimestre,Semestre
count,14216,14216.000000,14216.000000,14216.000000,14216.000000,14216.000000,14216,14216.000000,14216.000000,14216.000000,14216.000000,14216.000000,14216.000000,14216.000000
mean,2020-12-14 04:19:44.890264576,2.078579,-12.810718,45.403393,34.274064,2.197383,2020-12-14 04:19:44.890264576,2020.443374,6.642656,15.367614,13.367262,3.045442,2.542839,1.474958
min,2018-05-10 23:19:00,0.143478,-13.507734,44.653645,-1.960000,1.000000,2018-05-10 23:19:00,2018.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
25%,2019-08-21 23:43:45,1.477645,-12.835257,45.339040,30.000000,1.000000,2019-08-21 23:43:45,2019.000000,4.000000,7.000000,4.000000,1.000000,2.000000,1.000000
50%,2020-05-15 09:16:30,1.925589,-12.813657,45.365162,34.307955,2.000000,2020-05-15 09:16:30,2020.000000,7.000000,15.000000,17.000000,3.000000,3.000000,1.000000
75%,2021-12-21 00:15:45,2.567956,-12.793029,45.429007,38.777344,3.000000,2021-12-21 00:15:45,2021.000000,9.000000,23.000000,21.000000,5.000000,3.000000,2.000000
max,2025-05-08 20:34:00,5.900000,-11.714700,46.404640,184.508484,5.000000,2025-05-08 20:34:00,2025.000000,12.000000,31.000000,23.000000,6.000000,4.000000,2.000000
std,NaN,0.822759,0.044781,0.107224,7.364282,1.246709,NaN,1.747848,3.273409,8.966734,8.549266,2.052985,1.075219,0.499390



Valeurs manquantes par colonne:


Date           0
Magnitude      0
Latitude       0
Longitude      0
Profondeur     0
origine        0
Date_orig      0
Date_dt        0
Annee          0
Mois           0
Jour           0
Heure          0
JourSemaine    0
Trimestre      0
Semestre       0
dtype: int64


Période couverte: de 2018-05-10 23:19:00 à 2025-05-08 20:34:00
Durée: 2554 jours

--- INTERFACE D'ANALYSE SPATIO-TEMPORELLE DES SEISMES ---


Output()

In [ ]:
######### Analyses des tendances seismiques   ##############

import pandas as pd
from datetime import datetime
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.nonparametric.smoothers_lowess import lowess
import ipywidgets as widgets
from IPython.display import display, clear_output
import calendar
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')

# Charger les données
# Fonction robuste pour analyser les dates avec plusieurs formats
def parse_date_flexible(date_str):
    """
    Fonction robuste pour parser différents formats de date
    Gère spécifiquement les formats identifiés dans vos données comme 
    "08/05/2025 01:04" et "14/2/25 6:08"
    """
    if pd.isna(date_str):
        return None
        
    try:
        # Si c'est déjà un datetime
        if isinstance(date_str, datetime):
            return date_str
            
        # Nettoyer la chaîne si nécessaire
        date_str = str(date_str).strip()
        
        # Formats détectés dans vos données
        formats_to_try = [
            '%d/%m/%Y %H:%M',    # Pour "08/05/2025 01:04" (jour/mois/année sur 4 chiffres)
            '%d/%m/%y %H:%M',    # Pour "08/05/25 01:04" (jour/mois/année sur 2 chiffres)
            '%d/%m/%Y %H:%M:%S',
            '%d/%m/%y %H:%M:%S',
            # Formats sans zéros de remplissage (pour "14/2/25 6:08")
            '%d/%-m/%y %-H:%M',  # Linux/Mac
            '%d/%m/%y %H:%M',    # Windows (essai alternatif)
            # Autres formats possibles
            '%Y-%m-%dT%H:%M:%S.%f',  # Format ISO avec millisecondes
            '%Y-%m-%dT%H:%M:%S',     # Format ISO sans millisecondes
            '%Y-%m-%d %H:%M:%S'
        ]
        
        # Essayer tous les formats explicites d'abord
        for fmt in formats_to_try:
            try:
                # Adaptation pour Windows qui ne supporte pas %-
                if '%-' in fmt and '/' in date_str:
                    # Extraire les parties de la date pour un traitement manuel
                    parts = date_str.split()
                    if len(parts) == 2:  # Format "14/2/25 6:08"
                        date_part = parts[0]
                        time_part = parts[1]
                        
                        # Découper les composants de la date
                        day, month, year = date_part.split('/')
                        hour, minute = time_part.split(':')
                        
                        # Convertir en nombres
                        day = int(day)
                        month = int(month)
                        
                        # Déterminer si l'année est sur 2 ou 4 chiffres
                        if len(year) == 2:
                            # Convertir année sur 2 chiffres (20xx)
                            year = 2000 + int(year)
                        else:
                            year = int(year)
                            
                        hour = int(hour)
                        minute = int(minute)
                        
                        # Créer l'objet datetime
                        return datetime(year, month, day, hour, minute)
                else:
                    return datetime.strptime(date_str, fmt)
            except:
                continue
        
        # Si aucun format explicite ne fonctionne, essayer avec dateutil.parser
        try:
            # Pour les formats JJ/MM/YYYY ou JJ/MM/YY, utiliser dayfirst=True
            if '/' in date_str and len(date_str.split('/')[0]) <= 2:
                # Exemple: "14/2/25 6:08" ou "08/05/2025 01:04"
                return parser.parse(date_str, dayfirst=True)
            else:
                return parser.parse(date_str)
        except Exception as e:
            print(f"Échec du parsing avec dateutil: {date_str} - {e}")
    
    except Exception as e:
        print(f"Erreur lors du parsing de la date '{date_str}': {e}")
    
    return None

# Charger les données
def charger_donnees():
    """Charge et prépare les données sismiques avec gestion robuste des dates"""
    # Essayer d'abord avec le nom de fichier original
    try:
        df = pd.read_csv('NewDataseisme_corrige.csv', sep=';')
    except FileNotFoundError:
        # Essayer avec le nom alternatif qui apparaît dans le code
        try:
            df = pd.read_csv('NewData-seisme_corrige.csv', sep=';')
        except FileNotFoundError:
            # Si les deux échouent, essayer sans tiret et avec séparateur virgule
            try:
                df = pd.read_csv('NewDataseisme_corrige.csv', sep=',')
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier: {e}")
                print("Fichiers essayés: 'NewDataseisme_corrige.csv' et 'NewData-seisme_corrige.csv'")
                print("Vérifiez que le fichier existe et que le nom est correct.")
                return None
    
    # Afficher un aperçu
    print("Aperçu des données:")
    display(df.head())
    
    # Vérifier le format des dates (afficher quelques exemples)
    print("\nExemples de format de date dans les données:")
    if 'Date' in df.columns:
        date_samples = df['Date'].head(5).tolist()
        for sample in date_samples:
            print(sample)
    
    # Convertir la colonne date avec notre fonction robuste
    if 'Date' in df.columns:
        print("\nConversion des dates avec parse_date_flexible...")
        
        # Créer une nouvelle colonne pour les dates converties
        df['Date_dt'] = df['Date'].apply(parse_date_flexible)
        
        # Vérifier le succès de la conversion
        success_count = df['Date_dt'].notna().sum()
        fail_count = df['Date_dt'].isna().sum()
        
        print(f"Conversion des dates: {success_count} succès, {fail_count} échecs")
        
        if fail_count > 0:
            print(f"Attention: {fail_count} dates n'ont pas pu être converties et seront traitées comme manquantes")
        
        # Si la conversion de date a réussi, utiliser la colonne Date_dt
        if success_count > 0:
            # Remplacer la colonne Date par Date_dt
            df['Date_orig'] = df['Date']  # Garder l'original par sécurité
            df['Date'] = df['Date_dt']
        else:
            print("ERREUR: Aucune date n'a pu être convertie correctement")
            return None
    else:
        print("ERREUR: Colonne 'Date' non trouvée dans les données")
        return None
    
    # Corriger le format des nombres (virgule à point)
    for col in ['Magnitude', 'Latitude', 'Longitude', 'Profondeur']:
        if col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].str.replace(',', '.').astype(float)
    
    # Extraire les composantes temporelles
    df['Annee'] = df['Date'].dt.year
    df['Mois'] = df['Date'].dt.month
    df['Jour'] = df['Date'].dt.day
    df['Heure'] = df['Date'].dt.hour
    df['JourSemaine'] = df['Date'].dt.dayofweek  # 0=Lundi, 6=Dimanche
    df['Trimestre'] = df['Date'].dt.quarter
    
    # Ajouter ces composantes supplémentaires spécifiques à l'analyse des tendances
    try:
        df['Semaine'] = df['Date'].dt.isocalendar().week
    except AttributeError:
        # Pour les versions plus anciennes de pandas
        df['Semaine'] = df['Date'].dt.week
    
    df['JourAnnee'] = df['Date'].dt.dayofyear
    
    # Définir les saisons: Printemps (3-5), Été (6-8), Automne (9-11), Hiver (12,1,2)
    seasons = {
        'Hiver': [12, 1, 2],
        'Printemps': [3, 4, 5],
        'Été': [6, 7, 8],
        'Automne': [9, 10, 11]
    }
    
    # Créer une colonne 'Saison'
    def get_season(month):
        for season, months in seasons.items():
            if month in months:
                return season
    
    df['Saison'] = df['Mois'].apply(get_season)
    
    # Nettoyer les colonnes (supprimer les caractères \r si présents)
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('\r', '')
    
    print(f"Données chargées: {len(df)} enregistrements")
    
    if not df['Date'].isna().all():
        print(f"Période couverte: {df['Date'].min().date()} à {df['Date'].max().date()}")
    
    return df

# Fonction pour appliquer les filtres aux données
def appliquer_filtres(df, annee_range, mois_selected, magnitude_range, profondeur_range):
    """Applique les filtres sélectionnés au dataframe"""
    df_filtered = df.copy()
    
    # Filtrer par année
    df_filtered = df_filtered[(df_filtered['Annee'] >= annee_range[0]) & 
                            (df_filtered['Annee'] <= annee_range[1])]
    
    # Filtrer par mois
    df_filtered = df_filtered[df_filtered['Mois'].isin(mois_selected)]
    
    # Filtrer par magnitude
    df_filtered = df_filtered[(df_filtered['Magnitude'] >= magnitude_range[0]) & 
                            (df_filtered['Magnitude'] <= magnitude_range[1])]
    
    # Filtrer par profondeur
    df_filtered = df_filtered[(df_filtered['Profondeur'] >= profondeur_range[0]) & 
                            (df_filtered['Profondeur'] <= profondeur_range[1])]
    
    return df_filtered

# 1. Analyse des tendances saisonnières
# Remplacez la fonction analyser_tendances_saisonnieres par celle-ci
def analyser_tendances_saisonnieres(df_filtered):
    """Analyse les tendances saisonnières dans les données sismiques"""
    # Noms des mois
    mois_noms = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 
                 'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
    
    # 1. Nombre de séismes par mois de l'année (tous les ans confondus)
    plt.figure(figsize=(14, 6))
    mois_counts = df_filtered.groupby('Mois').size()
    
    # Créer un dictionnaire pour tous les mois (même ceux sans données)
    mois_dict = {i: 0 for i in range(1, 13)}
    for mois, count in mois_counts.items():
        mois_dict[mois] = count
    
    # Utiliser le dictionnaire pour le graphique
    plt.bar(range(1, 13), [mois_dict[i] for i in range(1, 13)])
    plt.title('Nombre de séismes par mois (toutes années confondues)')
    plt.xlabel('Mois')
    plt.ylabel('Nombre de séismes')
    plt.xticks(range(1, 13), mois_noms, rotation=45)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Test statistique pour vérifier si la distribution mensuelle est uniforme
    if len(mois_counts) == 12:  # Seulement si tous les mois sont présents
        chi2, p = stats.chisquare(mois_counts)
        print(f"Test Chi² d'uniformité de la distribution mensuelle: chi²={chi2:.2f}, p-value={p:.4f}")
        if p < 0.05:
            print("Il existe une variation saisonnière statistiquement significative dans le nombre de séismes par mois.")
        else:
            print("La distribution mensuelle des séismes semble uniforme (pas de tendance saisonnière significative).")
    else:
        print("Test Chi² non effectué car tous les mois ne sont pas représentés dans les données filtrées.")
    
    # 2. Heatmap des séismes par mois et par année si plusieurs années
    if len(df_filtered['Annee'].unique()) > 1:
        plt.figure(figsize=(14, 8))
        
        # Créer un DataFrame avec tous les mois et années possibles
        annees = sorted(df_filtered['Annee'].unique())
        
        # Préparer les données pour la heatmap
        try:
            heatmap_data = df_filtered.groupby(['Annee', 'Mois']).size().unstack(fill_value=0)
            
            # S'assurer que toutes les colonnes (mois) existent
            for m in range(1, 13):
                if m not in heatmap_data.columns:
                    heatmap_data[m] = 0
            
            # Trier les colonnes pour avoir l'ordre des mois correct
            heatmap_data = heatmap_data.reindex(sorted(heatmap_data.columns), axis=1)
            
            # Création d'une colormap personnalisée
            colors = ["#f7fbff", "#deebf7", "#c6dbef", "#9ecae1", "#6baed6", "#4292c6", "#2171b5", "#08519c", "#08306b"]
            cmap = LinearSegmentedColormap.from_list("custom_blues", colors)
            
            # Afficher la heatmap
            ax = sns.heatmap(heatmap_data, cmap=cmap, annot=True, fmt="d", linewidths=.5)
            plt.title('Nombre de séismes par mois et par année')
            plt.xlabel('Mois')
            plt.ylabel('Année')
            
            # Configurer les étiquettes d'axe
            ax.set_xticklabels([mois_noms[i-1] for i in heatmap_data.columns], rotation=45)
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print(f"Impossible de créer la heatmap: {e}")
            print("Cela peut être dû à un nombre insuffisant de données après filtrage.")
    
    # 3. Évolution de la magnitude moyenne par mois
    plt.figure(figsize=(14, 6))
    mag_means = df_filtered.groupby('Mois')['Magnitude'].mean()
    
    # Créer un dictionnaire pour tous les mois (même ceux sans données)
    mag_dict = {i: 0 for i in range(1, 13)}
    for mois, mean in mag_means.items():
        mag_dict[mois] = mean
    
    # Utiliser uniquement les mois pour lesquels on a des données
    mois_avec_donnees = sorted(mag_means.index)
    plt.bar(mois_avec_donnees, [mag_dict[m] for m in mois_avec_donnees], color='orange')
    plt.title('Magnitude moyenne des séismes par mois')
    plt.xlabel('Mois')
    plt.ylabel('Magnitude moyenne')
    plt.xticks(mois_avec_donnees, [mois_noms[i-1] for i in mois_avec_donnees], rotation=45)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 4. Analyse par saison
    plt.figure(figsize=(12, 5))
    season_counts = df_filtered.groupby('Saison').size()
    
    # Réorganiser les saisons dans l'ordre chronologique
    season_order = ['Hiver', 'Printemps', 'Été', 'Automne']
    season_counts = season_counts.reindex([s for s in season_order if s in season_counts.index])
    
    plt.bar(season_counts.index, season_counts.values, color='purple')
    plt.title('Nombre de séismes par saison')
    plt.xlabel('Saison')
    plt.ylabel('Nombre de séismes')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Afficher les statistiques par saison
    if len(season_counts) > 0:
        print("\nRépartition des séismes par saison:")
        for saison, count in season_counts.items():
            print(f"{saison}: {count} séismes ({count/len(df_filtered)*100:.1f}%)")

# 2. Analyse des tendances journalières
def analyser_tendances_journalieres(df_filtered):
    """Analyse les tendances journalières dans les données sismiques"""
    
    # 1. Distribution des séismes par heure
    plt.figure(figsize=(14, 6))
    heure_counts = df_filtered.groupby('Heure').size()
    
    plt.bar(heure_counts.index, heure_counts.values, color='darkblue')
    plt.title('Nombre de séismes par heure de la journée')
    plt.xlabel('Heure')
    plt.ylabel('Nombre de séismes')
    plt.xticks(range(0, 24))
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Diviser la journée en 4 périodes de 6h
    periodes = {
        'Nuit (0h-6h)': list(range(0, 6)),
        'Matin (6h-12h)': list(range(6, 12)),
        'Après-midi (12h-18h)': list(range(12, 18)),
        'Soir (18h-24h)': list(range(18, 24))
    }
    
    periode_counts = {}
    for nom, heures in periodes.items():
        periode_counts[nom] = df_filtered[df_filtered['Heure'].isin(heures)].shape[0]
    
    plt.figure(figsize=(12, 5))
    plt.bar(periode_counts.keys(), periode_counts.values(), color='navy')
    plt.title('Nombre de séismes par période de la journée')
    plt.xlabel('Période')
    plt.ylabel('Nombre de séismes')
    plt.xticks(rotation=45)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 2. Distribution par jour de la semaine
    plt.figure(figsize=(14, 6))
    jours_semaine = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    jour_counts = df_filtered.groupby('JourSemaine').size()
    
    # Créer un dictionnaire pour tous les jours (même ceux sans données)
    jours_dict = {i: 0 for i in range(7)}
    for jour, count in jour_counts.items():
        jours_dict[jour] = count
    
    plt.bar(range(7), [jours_dict.get(i, 0) for i in range(7)], color='darkgreen')
    plt.title('Nombre de séismes par jour de la semaine')
    plt.xlabel('Jour')
    plt.ylabel('Nombre de séismes')
    plt.xticks(range(7), jours_semaine)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Comparer jours de semaine vs weekend
    semaine = df_filtered[df_filtered['JourSemaine'] < 5].shape[0]
    weekend = df_filtered[df_filtered['JourSemaine'] >= 5].shape[0]
    
    plt.figure(figsize=(10, 5))
    plt.bar(['Jours de semaine (Lun-Ven)', 'Weekend (Sam-Dim)'], [semaine, weekend], color=['blue', 'red'])
    plt.title('Nombre de séismes : Jours de semaine vs Weekend')
    plt.ylabel('Nombre de séismes')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Afficher les statistiques
    total = semaine + weekend
    if total > 0:
        print("\nRépartition des séismes entre semaine et weekend:")
        print(f"Jours de semaine (Lun-Ven): {semaine} séismes ({semaine/total*100:.1f}%)")
        print(f"Weekend (Sam-Dim): {weekend} séismes ({weekend/total*100:.1f}%)")
        print(f"Rapport observé: {weekend/semaine:.2f} (weekend/semaine)")
        print(f"Rapport attendu si distribution uniforme: {2/5:.2f} (2 jours / 5 jours)")

# 3. Analyse des tendances à long terme
def analyser_tendances_long_terme(df_filtered):
    """Analyse les tendances à long terme dans les données sismiques"""
    
    # Vérifier si nous avons suffisamment d'années pour cette analyse
    annees_uniques = df_filtered['Annee'].unique()
    if len(annees_uniques) < 2:
        print("Cette analyse nécessite des données sur au moins 2 années différentes.")
        print(f"Années disponibles dans la sélection: {', '.join(map(str, sorted(annees_uniques)))}")
        return
    
    # Regrouper les données par mois pour l'analyse des séries temporelles
    df_mensuel = df_filtered.groupby(pd.Grouper(key='Date', freq='M')).agg({
        'Magnitude': ['count', 'mean', 'max'],
        'Profondeur': 'mean'
    })
    
    df_mensuel.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_mensuel.columns.values]
    df_mensuel.rename(columns={'Magnitude_count': 'Nombre_Seismes'}, inplace=True)
    
    # 1. Nombre de séismes par mois au fil du temps
    plt.figure(figsize=(14, 6))
    plt.plot(df_mensuel.index, df_mensuel['Nombre_Seismes'], marker='o', linestyle='-', color='blue')
    
    # Ajout d'une ligne de tendance avec régression linéaire
    if len(df_mensuel) > 1:
        X = np.arange(len(df_mensuel)).reshape(-1, 1)
        y = df_mensuel['Nombre_Seismes'].values
        
        model = stats.linregress(X.flatten(), y)
        trend_line = model.slope * X.flatten() + model.intercept
        plt.plot(df_mensuel.index, trend_line, color='red', linestyle='--', 
                label=f'Tendance (pente={model.slope:.4f}, p={model.pvalue:.4f})')
        
        if model.pvalue < 0.05:
            trend_direction = "augmentation" if model.slope > 0 else "diminution"
            print(f"Il existe une tendance significative à la {trend_direction} du nombre de séismes au fil du temps.")
            print(f"Pente de la tendance: {model.slope:.4f} séismes/mois, p-value: {model.pvalue:.4f}")
        else:
            print("Aucune tendance significative n'a été détectée dans le nombre de séismes au fil du temps.")
    
    # Ajouter une courbe lissée (moyenne mobile)
    if len(df_mensuel) >= 6:  # Au moins 6 mois pour calculer une moyenne mobile
        window_size = min(6, len(df_mensuel) // 2)  # Utiliser un minimum de 6 mois ou la moitié des données disponibles
        rolling_mean = df_mensuel['Nombre_Seismes'].rolling(window=window_size, center=True).mean()
        plt.plot(df_mensuel.index, rolling_mean, color='green', linestyle='-.',
                label=f'Moyenne mobile ({window_size} mois)')
    
    plt.title('Évolution du nombre de séismes par mois')
    plt.xlabel('Date')
    plt.ylabel('Nombre de séismes')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 2. Magnitude moyenne par mois au fil du temps
    plt.figure(figsize=(14, 6))
    plt.plot(df_mensuel.index, df_mensuel['Magnitude_mean'], marker='o', linestyle='-', color='orange')
    
    # Ajouter une ligne de tendance
    if len(df_mensuel) > 1:
        X = np.arange(len(df_mensuel)).reshape(-1, 1)
        y = df_mensuel['Magnitude_mean'].values
        model_mag = stats.linregress(X.flatten(), y)
        trend_line_mag = model_mag.slope * X.flatten() + model_mag.intercept
        plt.plot(df_mensuel.index, trend_line_mag, color='red', linestyle='--', 
                label=f'Tendance (pente={model_mag.slope:.4f}, p={model_mag.pvalue:.4f})')
        
        if model_mag.pvalue < 0.05:
            trend_direction = "augmentation" if model_mag.slope > 0 else "diminution"
            print(f"Il existe une tendance significative à la {trend_direction} de la magnitude moyenne au fil du temps.")
            print(f"Pente de la tendance: {model_mag.slope:.4f} magnitude/mois, p-value: {model_mag.pvalue:.4f}")
        else:
            print("Aucune tendance significative n'a été détectée dans la magnitude moyenne au fil du temps.")
    
    plt.title('Évolution de la magnitude moyenne par mois')
    plt.xlabel('Date')
    plt.ylabel('Magnitude moyenne')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 3. Analyse des tendances par année - tableau récapitulatif
    df_annuel = df_filtered.groupby('Annee').agg({
        'Magnitude': ['count', 'mean', 'max', 'min', 'std'],
        'Profondeur': ['mean', 'min', 'max', 'std']
    })
    
    df_annuel.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_annuel.columns.values]
    
    print("\nRésumé annuel de l'activité sismique:")
    display(df_annuel)

# 4. Analyse des cycles et périodicités
def analyser_cycles_periodicites(df_filtered):
    """Analyse les cycles et périodicités potentiels dans les données sismiques"""
    
    # Vérifier si nous avons suffisamment de données pour cette analyse
    if len(df_filtered) < 100:
        print("Cette analyse nécessite un grand nombre de données (>100 séismes).")
        print(f"Nombre d'enregistrements dans la sélection actuelle: {len(df_filtered)}")
        return
    
    # Créer une série temporelle pour l'analyse de périodicité
    # Agréger par jour pour avoir une série temporelle régulière
    ts_daily = df_filtered.groupby(df_filtered['Date'].dt.date).size()
    date_range = pd.date_range(start=ts_daily.index.min(), end=ts_daily.index.max())
    ts_daily = ts_daily.reindex(date_range, fill_value=0)
    
    # 1. Autocorrélation - détection de périodicité
    plt.figure(figsize=(14, 6))
    pd.plotting.autocorrelation_plot(ts_daily)
    plt.title('Autocorrélation du nombre de séismes par jour')
    plt.xlim(0, min(100, len(ts_daily)))  # Limiter à 100 lags pour mieux voir les cycles courts
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("L'autocorrélation permet de détecter des cycles périodiques dans les données.")
    print("Les pics dans le graphique suggèrent des périodicités possibles.")
    
    # 2. Analyse de la périodicité hebdomadaire
    plt.figure(figsize=(12, 5))
    jour_semaine_counts = df_filtered.groupby('JourSemaine').size()
    
    # Créer un dictionnaire pour tous les jours (même ceux sans données)
    jours_dict = {i: 0 for i in range(7)}
    for jour, count in jour_semaine_counts.items():
        jours_dict[jour] = count
    
    jours_semaine = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']
    plt.bar(range(7), [jours_dict.get(i, 0) for i in range(7)], color='purple')
    plt.title('Nombre de séismes par jour de la semaine (cycle hebdomadaire)')
    plt.xlabel('Jour de la semaine')
    plt.ylabel('Nombre de séismes')
    plt.xticks(range(7), jours_semaine)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 3. Analyse par saison (pour détecter des cycles annuels)
    if len(df_filtered['Annee'].unique()) >= 1:
        plt.figure(figsize=(14, 6))
        
        # Compter les séismes par jour de l'année
        jour_annee_counts = df_filtered.groupby('JourAnnee').size()
        
        plt.plot(jour_annee_counts.index, jour_annee_counts.values)
        plt.title('Nombre de séismes par jour de l\'année (cycles annuels)')
        plt.xlabel('Jour de l\'année')
        plt.ylabel('Nombre de séismes')
        
        # Répartir les jours par mois pour l'affichage
        mois_jours = [(1, 31), (2, 28), (3, 31), (4, 30), (5, 31), (6, 30), 
                    (7, 31), (8, 31), (9, 30), (10, 31), (11, 30), (12, 31)]
        mois_limites = [1]
        for m, j in mois_jours:
            mois_limites.append(mois_limites[-1] + j)
        
        # Ajouter des lignes verticales pour séparer les mois
        mois_noms = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 
                    'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
        for i, limite in enumerate(mois_limites[:-1]):
            plt.axvline(x=limite, color='gray', linestyle='--', alpha=0.5)
            if jour_annee_counts.max() > 0:
                plt.text(limite + (mois_limites[i+1] - limite)/2, jour_annee_counts.max() * 0.9, 
                        mois_noms[i], ha='center')
        
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

# Création du Dashboard
def creer_dashboard():
    # Charger les données
    df = charger_donnees()
    
    # Créer les widgets pour les filtres
    output_dashboard = widgets.Output()
    
    # Filtres temporels
    annees = sorted(df['Annee'].unique())
    mois = list(range(1, 13))
    mois_noms = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 
                'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
    mois_dict = {i+1: nom for i, nom in enumerate(mois_noms)}
    
    # Filtres magnitude et profondeur
    min_mag = float(df['Magnitude'].min())
    max_mag = float(df['Magnitude'].max())
    min_prof = float(df['Profondeur'].min())
    max_prof = float(df['Profondeur'].max())
    
    # Création des widgets
    annee_slider = widgets.IntRangeSlider(
        value=[annees[0], annees[-1]],
        min=annees[0],
        max=annees[-1],
        step=1,
        description='Années:',
        continuous_update=False,
        layout=widgets.Layout(width='70%')
    )
    
    mois_checkbox = widgets.SelectMultiple(
        options=[(mois_dict[m], m) for m in mois],
        value=mois,
        description='Mois:',
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    magnitude_slider = widgets.FloatRangeSlider(
        value=[min_mag, max_mag],
        min=min_mag,
        max=max_mag,
        step=0.1,
        description='Magnitude:',
        continuous_update=False,
        layout=widgets.Layout(width='70%')
    )
    
    profondeur_slider = widgets.FloatRangeSlider(
        value=[min_prof, max_prof],
        min=min_prof,
        max=max_prof,
        step=5,
        description='Profondeur:',
        continuous_update=False,
        layout=widgets.Layout(width='70%')
    )
    
    # Types d'analyse
    analyse_type = widgets.RadioButtons(
        options=['Tendances saisonnières', 'Tendances journalières', 
                'Tendances à long terme', 'Cycles et périodicités'],
        description='Type d\'analyse:',
        layout=widgets.Layout(width='50%')
    )
    
    filtrer_button = widgets.Button(
        description='Analyser les tendances',
        button_style='primary',
        tooltip='Cliquez pour analyser les tendances',
        layout=widgets.Layout(width='200px')
    )
    
    # Fonction principale pour mettre à jour l'analyse
    def update_dashboard(b):
        with output_dashboard:
            clear_output(wait=True)
            
            # Récupérer les valeurs des filtres
            annee_range = annee_slider.value
            mois_selected = mois_checkbox.value
            magnitude_range = magnitude_slider.value
            profondeur_range = profondeur_slider.value
            
            # Appliquer les filtres
            df_filtered = appliquer_filtres(df, annee_range, mois_selected, magnitude_range, profondeur_range)
            
            print(f"Données filtrées: {len(df_filtered)} séismes sur {len(df)} ({len(df_filtered)/len(df)*100:.1f}%)")
            
            if len(df_filtered) == 0:
                print("Aucune donnée ne correspond aux critères de filtrage!")
                return
            
            # Effectuer l'analyse sélectionnée
            analysis_type = analyse_type.value
            
            if analysis_type == 'Tendances saisonnières':
                analyser_tendances_saisonnieres(df_filtered)
                
            elif analysis_type == 'Tendances journalières':
                analyser_tendances_journalieres(df_filtered)
                
            elif analysis_type == 'Tendances à long terme':
                analyser_tendances_long_terme(df_filtered)
                
            elif analysis_type == 'Cycles et périodicités':
                analyser_cycles_periodicites(df_filtered)
    
    # Connecter le bouton à la fonction d'actualisation
    filtrer_button.on_click(update_dashboard)
    
     # Afficher l'interface
    print("\n--- DASHBOARD D'ANALYSE DES TENDANCES SISMIQUES ---")
    controls = widgets.VBox([
        widgets.HBox([widgets.Label('Filtres temporels', style={'font_weight': 'bold'})]),
        widgets.HBox([annee_slider]),
        widgets.HBox([mois_checkbox]),
        widgets.HBox([widgets.Label('Filtres de caractéristiques', style={'font_weight': 'bold'})]),
        widgets.HBox([magnitude_slider]),
        widgets.HBox([profondeur_slider]),
        widgets.HBox([widgets.Label('Type d\'analyse', style={'font_weight': 'bold'})]),
        widgets.HBox([analyse_type]),
        widgets.HBox([filtrer_button])
    ])
    
    display(controls)
    display(output_dashboard)
    
    # Lancer l'analyse avec les filtres par défaut
    update_dashboard(None)

# Lancer le dashboard
if __name__ == "__main__":
    creer_dashboard()

Aperçu des données:


,Date,Magnitude,Latitude,Longitude,Profondeur,origine
0,08/05/2025 20:34,"1,998769442","-12,750200","45,561000","51,54",5
1,08/05/2025 20:27,"2,374112553","-12,788200","45,619300","47,24",5
2,08/05/2025 19:16,"1,667391108","-12,566000","45,175300","38,39",5
3,08/05/2025 01:04,"2,025011775","-12,805300","45,580500","49,82",5
4,07/05/2025 18:46,"1,282582543","-12,805500","45,347200","43,46",5



Exemples de format de date dans les données:
08/05/2025 20:34
08/05/2025 20:27
08/05/2025 19:16
08/05/2025 01:04
07/05/2025 18:46

Conversion des dates avec parse_date_flexible...
Conversion des dates: 14216 succès, 0 échecs
Données chargées: 14216 enregistrements
Période couverte: 2018-05-10 à 2025-05-08

--- DASHBOARD D'ANALYSE DES TENDANCES SISMIQUES ---


Output()

In [1]:
######## ANALYSE DES CARACTÉRISTIQUES SISMIQUES   ##########



import pandas as pd
from datetime import datetime
from dateutil import parser
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')


def parse_date_flexible(date_str):
    """
    Fonction robuste pour parser différents formats de date
    Gère spécifiquement les formats identifiés dans vos données comme 
    "08/05/2025 01:04" et "14/2/25 6:08"
    """
    from datetime import datetime
    from dateutil import parser
    import pandas as pd
    
    if pd.isna(date_str):
        return None
        
    try:
        # Si c'est déjà un datetime
        if isinstance(date_str, datetime):
            return date_str
            
        # Nettoyer la chaîne si nécessaire
        date_str = str(date_str).strip()
        
        # Formats détectés dans vos données
        formats_to_try = [
            '%d/%m/%Y %H:%M',    # Pour "08/05/2025 01:04" (jour/mois/année sur 4 chiffres)
            '%d/%m/%y %H:%M',    # Pour "08/05/25 01:04" (jour/mois/année sur 2 chiffres)
            '%d/%m/%Y %H:%M:%S',
            '%d/%m/%y %H:%M:%S',
            # Formats sans zéros de remplissage (pour "14/2/25 6:08")
            '%d/%-m/%y %-H:%M',  # Linux/Mac
            '%d/%m/%y %H:%M',    # Windows (essai alternatif)
            # Autres formats possibles
            '%Y-%m-%dT%H:%M:%S.%f',  # Format ISO avec millisecondes
            '%Y-%m-%dT%H:%M:%S',     # Format ISO sans millisecondes
            '%Y-%m-%d %H:%M:%S'
        ]
        
        # Essayer tous les formats explicites d'abord
        for fmt in formats_to_try:
            try:
                # Adaptation pour Windows qui ne supporte pas %-
                if '%-' in fmt and '/' in date_str:
                    # Extraire les parties de la date pour un traitement manuel
                    parts = date_str.split()
                    if len(parts) == 2:  # Format "14/2/25 6:08"
                        date_part = parts[0]
                        time_part = parts[1]
                        
                        # Découper les composants de la date
                        day, month, year = date_part.split('/')
                        hour, minute = time_part.split(':')
                        
                        # Convertir en nombres
                        day = int(day)
                        month = int(month)
                        
                        # Déterminer si l'année est sur 2 ou 4 chiffres
                        if len(year) == 2:
                            # Convertir année sur 2 chiffres (20xx)
                            year = 2000 + int(year)
                        else:
                            year = int(year)
                            
                        hour = int(hour)
                        minute = int(minute)
                        
                        # Créer l'objet datetime
                        return datetime(year, month, day, hour, minute)
                else:
                    return datetime.strptime(date_str, fmt)
            except:
                continue
        
        # Si aucun format explicite ne fonctionne, essayer avec dateutil.parser
        try:
            # Pour les formats JJ/MM/YYYY ou JJ/MM/YY, utiliser dayfirst=True
            if '/' in date_str and len(date_str.split('/')[0]) <= 2:
                # Exemple: "14/2/25 6:08" ou "08/05/2025 01:04"
                return parser.parse(date_str, dayfirst=True)
            else:
                return parser.parse(date_str)
        except Exception as e:
            print(f"Échec du parsing avec dateutil: {date_str} - {e}")
    
    except Exception as e:
        print(f"Erreur lors du parsing de la date '{date_str}': {e}")
    
    return None

# Charger les données
# This is the corrected code section for handling negative depth values
# When loading and preparing the data, modify the code to use absolute values
# for profondeur (depth) before calculating potentiel_destructeur

def charger_donnees(nom_fichier=None):
    """Charge et prépare les données sismiques avec correction des profondeurs négatives"""
    import pandas as pd
    import numpy as np
    import warnings
    warnings.filterwarnings('ignore')
    
    # [Le reste du code de chargement reste inchangé...]
    
    # Liste des noms de fichiers à essayer
    fichiers_a_essayer = []
    if nom_fichier:
        fichiers_a_essayer.append(nom_fichier)
    
    # Ajouter les variantes possibles du nom de fichier
    fichiers_a_essayer.extend([
        'NewDataseisme_corriger.csv',
        'NewDataseisme_corrige.csv',
        'NewData-seisme_corriger.csv',
        'NewData-seisme_corrige.csv'
    ])
    
    # Essayer de charger chaque fichier avec différents séparateurs
    df = None
    for fichier in fichiers_a_essayer:
        try:
            print(f"Tentative de chargement du fichier: {fichier}")
            df = pd.read_csv(fichier, sep=';')
            print(f"Fichier chargé avec succès: {fichier}")
            break
        except FileNotFoundError:
            try:
                df = pd.read_csv(fichier, sep=',')
                print(f"Fichier chargé avec succès (séparateur virgule): {fichier}")
                break
            except FileNotFoundError:
                print(f"Fichier non trouvé: {fichier}")
                continue
        except Exception as e:
            print(f"Erreur lors du chargement de {fichier}: {e}")
            continue
    
    # Si aucun fichier n'a pu être chargé, créer des données factices pour démonstration
    if df is None:
        print("\nAucun fichier de données trouvé. Création de données de démonstration...")
        # Créer des données aléatoires pour la démonstration
        np.random.seed(42)  # Pour reproductibilité
        n_samples = 1000
        
        # Dates entre 2018 et 2025
        start_date = pd.Timestamp('2018-01-01')
        end_date = pd.Timestamp('2025-05-01')
        dates = pd.date_range(start=start_date, end=end_date, periods=n_samples)
        
        # Magnitudes entre 0.5 et 6.0, avec une distribution log-normale
        magnitudes = np.random.lognormal(mean=0.5, sigma=0.5, size=n_samples)
        magnitudes = np.clip(magnitudes, 0.5, 6.0)
        
        # Profondeurs entre 10 et 200 km
        profondeurs = np.random.lognormal(mean=3.5, sigma=0.7, size=n_samples)
        profondeurs = np.clip(profondeurs, 10, 200)
        
        # Créer le DataFrame
        df = pd.DataFrame({
            'Date': dates,
            'Magnitude': magnitudes,
            'Profondeur': profondeurs,
            'Latitude': np.random.uniform(-12.8, -12.7, n_samples),
            'Longitude': np.random.uniform(45.1, 45.2, n_samples)
        })
        
        print("Données de démonstration créées avec succès.")
    
    # Afficher un aperçu
    print("\nAperçu des données:")
    print(df.head())
    
    # Corriger le format des nombres (virgule à point)
    for col in ['Magnitude', 'Latitude', 'Longitude', 'Profondeur']:
        if col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].str.replace(',', '.').astype(float)
    
    # Convertir la colonne de date avec notre fonction robuste
    if 'Date' in df.columns and not pd.api.types.is_datetime64_any_dtype(df['Date']):
        # Examiner les premières valeurs pour comprendre le format
        date_samples = df['Date'].head(5).tolist()
        print("\nExemples de format de date dans les données:")
        for sample in date_samples:
            print(sample)
        
        print("\nConversion des dates avec parse_date_flexible...")
        # Appliquer la fonction de conversion
        df['Date'] = df['Date'].apply(parse_date_flexible)
        
        # Vérifier le succès de la conversion
        success_count = df['Date'].notna().sum()
        fail_count = df['Date'].isna().sum()
        
        print(f"Conversion des dates: {success_count} succès, {fail_count} échecs")
        
        if fail_count > 0:
            print(f"Attention: {fail_count} dates n'ont pas pu être converties et seront traitées comme manquantes")
            
        if success_count == 0:
            print("ERREUR: Aucune date n'a pu être convertie correctement. Création de dates factices.")
            # Créer des dates factices
            df['Date'] = pd.date_range(start='2018-01-01', periods=len(df), freq='D')
            print("Dates factices créées pour permettre l'analyse.")
    
    # Extraire les composantes temporelles
    df['Annee'] = df['Date'].dt.year
    df['Mois'] = df['Date'].dt.month
    df['Jour'] = df['Date'].dt.day
    df['Heure'] = df['Date'].dt.hour
    
    # Catégoriser les séismes selon leur magnitude
    magnitude_categories = [
        (0, 2.5, 'Micro'),
        (2.5, 4.0, 'Faible'),
        (4.0, 5.0, 'Léger'),
        (5.0, 6.0, 'Modéré'),
        (6.0, 7.0, 'Fort'),
        (7.0, 8.0, 'Majeur'),
        (8.0, float('inf'), 'Grand')
    ]
    
    def categorize_magnitude(mag):
        for low, high, category in magnitude_categories:
            if low <= mag < high:
                return category
        return 'Inconnu'
    
    df['Magnitude_Categorie'] = df['Magnitude'].apply(categorize_magnitude)
    
    # CORRECTION: Si des valeurs négatives de profondeur existent, prendre leur valeur absolue
    if (df['Profondeur'] < 0).any():
        print(f"\nAttention: {(df['Profondeur'] < 0).sum()} valeurs de profondeur négatives détectées.")
        print("Application de la valeur absolue pour ces valeurs...")
        df['Profondeur'] = df['Profondeur'].abs()
    
    # Catégoriser les séismes selon leur profondeur
    depth_categories = [
        (0, 70, 'Peu profond'),
        (70, 300, 'Intermédiaire'),
        (300, float('inf'), 'Profond')
    ]
    
    def categorize_depth(depth):
        for low, high, category in depth_categories:
            if low <= depth < high:
                return category
        return 'Inconnu'
    
    df['Profondeur_Categorie'] = df['Profondeur'].apply(categorize_depth)
    
    # Calcul de l'énergie libérée (formule approximative basée sur la magnitude)
    # Énergie en joules selon la formule E = 10^(1.5*M+4.8)
    df['Energie'] = 10**(1.5 * df['Magnitude'] + 4.8)
    
    # Estimation du potentiel destructeur (combinaison de magnitude et profondeur inverse)
    # Un séisme superficiel de grande magnitude est potentiellement plus destructeur
    # Formule simplifiée: Magnitude * (1 + 70/profondeur) 
    # CORRECTION: Nous nous assurons d'utiliser des profondeurs positives
    df['Potentiel_Destructeur'] = df['Magnitude'] * (1 + 70/df['Profondeur'])
    
    # Catégorisation du potentiel destructeur
    potentiel_categories = [
        (0, 3, 'Très faible'),
        (3, 6, 'Faible'),
        (6, 10, 'Modéré'),
        (10, 15, 'Élevé'),
        (15, float('inf'), 'Très élevé')
    ]
    
    def categorize_potentiel(pot):
        for low, high, category in potentiel_categories:
            if low <= pot < high:
                return category
        return 'Inconnu'
    
    df['Potentiel_Categorie'] = df['Potentiel_Destructeur'].apply(categorize_potentiel)
    
    # Nettoyer les colonnes (supprimer les caractères \r si présents)
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('\r', '')
    
    print(f"Données chargées: {len(df)} enregistrements")
    print(f"Période couverte: {df['Date'].min().date()} à {df['Date'].max().date()}")
    return df
    
# Cette fonction peut aussi être modifiée séparément si vous ne voulez pas modifier toute la fonction de chargement
def corriger_profondeurs_negatives(df):
    """Corrige les profondeurs négatives et recalcule le potentiel destructeur"""
    if (df['Profondeur'] < 0).any():
        print(f"\nAttention: {(df['Profondeur'] < 0).sum()} valeurs de profondeur négatives détectées.")
        print("Application de la valeur absolue pour ces valeurs...")
        
        # Créer une copie du dataframe pour ne pas modifier l'original
        df_corrige = df.copy()
        
        # Prendre la valeur absolue des profondeurs
        df_corrige['Profondeur'] = df_corrige['Profondeur'].abs()
        
        # Recatégoriser les profondeurs
        def categorize_depth(depth):
            if 0 <= depth < 70:
                return 'Peu profond'
            elif 70 <= depth < 300:
                return 'Intermédiaire'
            elif depth >= 300:
                return 'Profond'
            return 'Inconnu'
        
        df_corrige['Profondeur_Categorie'] = df_corrige['Profondeur'].apply(categorize_depth)
        
        # Recalculer le potentiel destructeur
        df_corrige['Potentiel_Destructeur'] = df_corrige['Magnitude'] * (1 + 70/df_corrige['Profondeur'])
        
        # Recatégoriser le potentiel destructeur
        def categorize_potentiel(pot):
            if 0 <= pot < 3:
                return 'Très faible'
            elif 3 <= pot < 6:
                return 'Faible'
            elif 6 <= pot < 10:
                return 'Modéré'
            elif 10 <= pot < 15:
                return 'Élevé'
            elif pot >= 15:
                return 'Très élevé'
            return 'Inconnu'
        
        df_corrige['Potentiel_Categorie'] = df_corrige['Potentiel_Destructeur'].apply(categorize_potentiel)
        
        return df_corrige
    else:
        print("Aucune profondeur négative détectée dans les données.")
        return df

# Version corrigée de la fonction preparer_donnees_clustering
def preparer_donnees_clustering(df):
    """Prépare les données pour le clustering avec gestion des valeurs extrêmes"""
    from sklearn.preprocessing import StandardScaler, RobustScaler
    import numpy as np
    
    print("Préparation des données pour le clustering...")
    
    # Sélectionner les caractéristiques pertinentes
    features = ['Magnitude', 'Profondeur', 'Potentiel_Destructeur', 'Energie']
    
    # Créer le jeu de données pour le clustering
    X = df[features].copy()
    
    # Vérifier et gérer les valeurs problématiques
    for feature in features:
        # Identifier les valeurs infinies ou NaN
        has_inf = np.isinf(X[feature]).any()
        has_nan = np.isnan(X[feature]).any()
        has_huge = (np.abs(X[feature]) > 1e100).any()
        
        if has_inf or has_nan or has_huge:
            print(f"  Détecté dans '{feature}': infinies={has_inf}, NaN={has_nan}, valeurs extrêmes={has_huge}")
            
            # Remplacer les valeurs infinies par la valeur max non-infinie * 2
            if has_inf:
                max_val = X[feature][~np.isinf(X[feature])].max()
                min_val = X[feature][~np.isinf(X[feature])].min()
                X.loc[X[feature] == np.inf, feature] = max_val * 2
                X.loc[X[feature] == -np.inf, feature] = min_val * 2
                print(f"  Valeurs infinies dans '{feature}' remplacées par {max_val*2}")
            
            # Remplacer les NaN par la médiane
            if has_nan:
                median = X[feature].median()
                X[feature] = X[feature].fillna(median)
                print(f"  NaN dans '{feature}' remplacés par la médiane: {median}")
            
            # Plafonner les valeurs extrêmes
            if has_huge:
                upper_limit = np.percentile(X[feature][~np.isinf(X[feature]) & ~np.isnan(X[feature])], 99)
                lower_limit = np.percentile(X[feature][~np.isinf(X[feature]) & ~np.isnan(X[feature])], 1)
                X.loc[X[feature] > upper_limit, feature] = upper_limit
                X.loc[X[feature] < lower_limit, feature] = lower_limit
                print(f"  Valeurs extrêmes dans '{feature}' plafonnées entre {lower_limit} et {upper_limit}")
    
    # Pour l'énergie qui peut avoir des valeurs très différentes, utiliser le logarithme
    if 'Energie' in features:
        # Garantir que l'énergie est positive
        min_energy = X['Energie'][X['Energie'] > 0].min()
        X.loc[X['Energie'] <= 0, 'Energie'] = min_energy
        # Appliquer la transformation logarithmique
        X['Energie'] = np.log10(X['Energie'])
        print("  Transformation logarithmique appliquée à l'énergie")
    
    # Traiter spécifiquement le potentiel destructeur
    if 'Potentiel_Destructeur' in features:
        # En cas de valeurs négatives, décaler pour que tout soit positif
        min_pd = X['Potentiel_Destructeur'].min()
        if min_pd < 0:
            X['Potentiel_Destructeur'] = X['Potentiel_Destructeur'] - min_pd + 1
            print(f"  Potentiel destructeur décalé pour éliminer les valeurs négatives (min = {min_pd})")
    
    print("Vérification finale des données:")
    for feature in features:
        if np.isinf(X[feature]).any() or np.isnan(X[feature]).any():
            print(f"  ATTENTION: {feature} contient encore des valeurs problématiques après nettoyage!")
    
    # Utiliser RobustScaler qui est moins sensible aux valeurs aberrantes
    print("Application de la mise à l'échelle robuste...")
    scaler = RobustScaler()
    try:
        X_scaled = scaler.fit_transform(X)
        print("Mise à l'échelle réussie!")
    except Exception as e:
        print(f"Erreur lors de la mise à l'échelle: {e}")
        # Plan B: Mise à l'échelle manuelle
        print("Tentative de mise à l'échelle manuelle...")
        X_scaled = np.zeros_like(X.values, dtype=float)
        for i, feature in enumerate(features):
            median = X[feature].median()
            iqr = X[feature].quantile(0.75) - X[feature].quantile(0.25)
            iqr = max(iqr, 1e-10)  # Éviter la division par zéro
            X_scaled[:, i] = (X[feature].values - median) / iqr
        print("Mise à l'échelle manuelle réussie!")
    
    return X_scaled, features

# Modification optionnelle pour la fonction analyser_clustering
def analyser_clustering(df_filtered):
    """Analyse les données sismiques par clustering avec gestion d'erreurs améliorée"""
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import KMeans
    import numpy as np
    
    print("\nAnalyse par clustering des données sismiques...")
    
    # Vérifier que nous avons assez de données
    if len(df_filtered) < 10:
        print("Pas assez de données pour le clustering (minimum 10 points requis).")
        return
    
    try:
        # Préparation des données
        X_scaled, features = preparer_donnees_clustering(df_filtered)
        
        # Vérifier qu'il n'y a pas de valeurs problématiques restantes
        if np.isnan(X_scaled).any() or np.isinf(X_scaled).any():
            print("ERREUR: Des valeurs infinies ou NaN persistent après prétraitement.")
            # Dernière tentative de nettoyage
            X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=10.0, neginf=-10.0)
            print("Remplacement forcé des valeurs problématiques.")
        
        # Déterminer le nombre optimal de clusters
        try:
            optimal_k = determiner_nombre_clusters(X_scaled)
        except Exception as e:
            print(f"Erreur lors de la détermination du nombre optimal de clusters: {e}")
            print("Utilisation de 3 clusters par défaut.")
            optimal_k = 3
        
        # Réaliser le clustering et analyser les résultats
        try:
            df_clusters = realiser_clustering(X_scaled, optimal_k, df_filtered, features)
            
            # Interprétation des clusters
            print("\nInterprétation suggérée des clusters:")
            
            # Trouver les caractéristiques distinctives de chaque cluster
            for cluster in range(optimal_k):
                cluster_data = df_clusters[df_clusters['Cluster'] == cluster]
                mean_mag = cluster_data['Magnitude'].mean()
                mean_depth = cluster_data['Profondeur'].mean()
                mean_pd = cluster_data['Potentiel_Destructeur'].mean()
                
                print(f"\nCluster {cluster} ({len(cluster_data)} séismes):")
                print(f"  Magnitude moyenne: {mean_mag:.2f}")
                print(f"  Profondeur moyenne: {mean_depth:.2f} km")
                print(f"  Potentiel destructeur moyen: {mean_pd:.2f}")
                
                # Caractérisation du cluster
                if mean_mag < 2.0 and mean_depth < 50:
                    print("  → Probablement des micro-séismes superficiels")
                elif mean_mag < 2.0 and mean_depth >= 50:
                    print("  → Probablement des micro-séismes profonds")
                elif mean_mag >= 2.0 and mean_mag < 4.0 and mean_depth < 50:
                    print("  → Probablement des séismes volcano-tectoniques superficiels")
                elif mean_mag >= 2.0 and mean_mag < 4.0 and mean_depth >= 50:
                    print("  → Probablement des séismes tectoniques régionaux")
                elif mean_mag >= 4.0:
                    print("  → Probablement des événements majeurs")
        except Exception as e:
            print(f"Erreur lors de la réalisation du clustering: {e}")
    except Exception as e:
        print(f"Erreur générale lors de l'analyse par clustering: {e}")
        print("Assurez-vous que vos données ne contiennent pas de valeurs aberrantes extrêmes.")

def determiner_nombre_clusters(X_scaled):
    """Détermine le nombre optimal de clusters avec gestion d'erreurs améliorée"""
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_score
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Définir la plage de nombre de clusters à tester
    k_range = range(2, 11)  # De 2 à 10 clusters
    
    # Initialiser les listes pour stocker les mesures
    inertias = []
    silhouette_scores = []
    
    print("Calcul des mesures pour différents nombres de clusters...")
    
    # Calculer l'inertie et le score de silhouette pour chaque valeur de k
    for k in k_range:
        print(f"  Évaluation pour k = {k}...")
        
        # Entraîner le modèle KMeans
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        
        # Stocker l'inertie
        inertias.append(kmeans.inertia_)
        
        # Calculer et stocker le score de silhouette (seulement pour k > 1)
        if k > 1:
            try:
                labels = kmeans.labels_
                score = silhouette_score(X_scaled, labels)
                silhouette_scores.append(score)
                print(f"    Score de silhouette: {score:.4f}")
            except Exception as e:
                print(f"    Erreur lors du calcul du score de silhouette pour k={k}: {e}")
                silhouette_scores.append(0)  # Valeur par défaut en cas d'erreur
    
    # Tracer la courbe du coude et les scores de silhouette
    plt.figure(figsize=(15, 6))
    
    # Courbe du coude
    plt.subplot(1, 2, 1)
    plt.plot(list(k_range), inertias, 'o-', color='blue')
    plt.xlabel('Nombre de clusters (k)')
    plt.ylabel('Inertie')
    plt.title('Méthode du coude')
    plt.grid(alpha=0.3)
    
    # Scores de silhouette
    plt.subplot(1, 2, 2)
    # CORRECTION ICI : k_range commence à 2, mais silhouette_scores commence à partir de k=2
    # Nous devons donc utiliser k_range[2-2:] pour aligner les dimensions
    plt.plot(list(k_range)[2-2:], silhouette_scores, 'o-', color='green')
    plt.xlabel('Nombre de clusters (k)')
    plt.ylabel('Score de silhouette')
    plt.title('Score de silhouette')
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Déterminer le nombre optimal de clusters
    if silhouette_scores:
        best_silhouette_idx = np.argmax(silhouette_scores)
        # Pour obtenir la valeur k correspondante, nous devons ajouter 2 à l'index
        # car silhouette_scores commence à k=2
        optimal_k = list(k_range)[best_silhouette_idx + (2-2)]
        print(f"Nombre optimal suggéré de clusters selon le score de silhouette: {optimal_k}")
    else:
        # Valeur par défaut si aucun score de silhouette n'a pu être calculé
        optimal_k = 3
        print(f"Impossible de déterminer le nombre optimal. Utilisation de la valeur par défaut: {optimal_k}")
    
    return optimal_k

def realiser_clustering(X_scaled, n_clusters, df, features):
    """Réalise le clustering et analyse les résultats"""
    from sklearn.cluster import KMeans
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    import numpy as np
    
    # Appliquer K-means avec le nombre optimal de clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df_clusters = df.copy()
    df_clusters['Cluster'] = kmeans.fit_predict(X_scaled)
    
    # Analyser les caractéristiques de chaque cluster
    cluster_stats = df_clusters.groupby('Cluster').agg({
        'Magnitude': ['mean', 'min', 'max', 'count'],
        'Profondeur': ['mean', 'min', 'max'],
        'Potentiel_Destructeur': ['mean', 'min', 'max'],
        'Energie': ['mean', 'sum']
    }).round(2)
    
    print("\nCaractéristiques des clusters identifiés:")
    display(cluster_stats)
    
    # Visualisation des clusters (en 2D en utilisant les 2 caractéristiques les plus importantes)
    plt.figure(figsize=(15, 10))
    
    # Visualiser les clusters pour différentes paires de caractéristiques
    feature_pairs = [
        ('Magnitude', 'Profondeur'),
        ('Magnitude', 'Potentiel_Destructeur'),
        ('Profondeur', 'Potentiel_Destructeur')
    ]
    
    for i, (feature1, feature2) in enumerate(feature_pairs):
        plt.subplot(2, 2, i+1)
        
        # Créer des couleurs distinctes pour chaque cluster
        scatter = plt.scatter(
            df_clusters[feature1], 
            df_clusters[feature2], 
            c=df_clusters['Cluster'], 
            cmap='viridis', 
            alpha=0.6,
            s=50
        )
        
        # Ajouter les centroïdes
        centers = kmeans.cluster_centers_
        feature1_idx = features.index(feature1)
        feature2_idx = features.index(feature2)
        plt.scatter(
            centers[:, feature1_idx], 
            centers[:, feature2_idx], 
            c='red', 
            marker='X', 
            s=200, 
            label='Centroïdes'
        )
        
        plt.xlabel(feature1)
        plt.ylabel(feature2)
        plt.title(f'Clusters: {feature1} vs {feature2}')
        plt.colorbar(scatter, label='Cluster')
        plt.legend()
        plt.grid(alpha=0.3)
    
    # Distribution des clusters dans le temps (si les données temporelles sont disponibles)
    if 'Date' in df_clusters.columns:
        plt.subplot(2, 2, 4)
        
        # Compter les séismes par cluster et par mois
        df_clusters['YearMonth'] = df_clusters['Date'].dt.to_period('M')
        time_clusters = df_clusters.groupby(['YearMonth', 'Cluster']).size().unstack().fillna(0)
        
        # Tracer la distribution temporelle
        time_clusters.plot(kind='area', stacked=True, colormap='viridis', alpha=0.7, ax=plt.gca())
        plt.xlabel('Mois')
        plt.ylabel('Nombre de séismes')
        plt.title('Évolution temporelle des clusters')
        plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Option: visualisation 3D
    try:
        from mpl_toolkits.mplot3d import Axes3D
        
        fig = plt.figure(figsize=(12, 10))
        ax = fig.add_subplot(111, projection='3d')
        
        scatter = ax.scatter(
            df_clusters['Magnitude'],
            df_clusters['Profondeur'],
            df_clusters['Potentiel_Destructeur'],
            c=df_clusters['Cluster'],
            cmap='viridis',
            s=50,
            alpha=0.6
        )
        
        ax.set_xlabel('Magnitude')
        ax.set_ylabel('Profondeur')
        ax.set_zlabel('Potentiel Destructeur')
        ax.set_title('Clusters sismiques en 3D')
        
        plt.colorbar(scatter, label='Cluster')
        plt.tight_layout()
        plt.show()
    except:
        print("Visualisation 3D non disponible")
    
    return df_clusters

# Nouvelle fonction d'analyse par clustering à ajouter à votre code
def analyser_clustering(df_filtered):
    """Analyse les données sismiques par clustering"""
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import KMeans
    
    print("\nAnalyse par clustering des données sismiques...")
    
    # Préparation des données
    X_scaled, features = preparer_donnees_clustering(df_filtered)
    
    # Déterminer le nombre optimal de clusters
    optimal_k = determiner_nombre_clusters(X_scaled)
    
    # Réaliser le clustering et analyser les résultats
    df_clusters = realiser_clustering(X_scaled, optimal_k, df_filtered, features)
    
    # Interprétation des clusters
    print("\nInterprétation suggérée des clusters:")
    
    # Trouver les caractéristiques distinctives de chaque cluster
    for cluster in range(optimal_k):
        cluster_data = df_clusters[df_clusters['Cluster'] == cluster]
        mean_mag = cluster_data['Magnitude'].mean()
        mean_depth = cluster_data['Profondeur'].mean()
        mean_pd = cluster_data['Potentiel_Destructeur'].mean()
        
        print(f"\nCluster {cluster} ({len(cluster_data)} séismes):")
        print(f"  Magnitude moyenne: {mean_mag:.2f}")
        print(f"  Profondeur moyenne: {mean_depth:.2f} km")
        print(f"  Potentiel destructeur moyen: {mean_pd:.2f}")
        
        # Caractérisation du cluster
        if mean_mag < 2.0 and mean_depth < 50:
            print("  → Probablement des micro-séismes superficiels")
        elif mean_mag < 2.0 and mean_depth >= 50:
            print("  → Probablement des micro-séismes profonds")
        elif mean_mag >= 2.0 and mean_mag < 4.0 and mean_depth < 50:
            print("  → Probablement des séismes volcano-tectoniques superficiels")
        elif mean_mag >= 2.0 and mean_mag < 4.0 and mean_depth >= 50:
            print("  → Probablement des séismes tectoniques régionaux")
        elif mean_mag >= 4.0:
            print("  → Probablement des événements majeurs")




# Fonction pour appliquer les filtres aux données
def appliquer_filtres(df, annee_range, mois_selected, magnitude_range, profondeur_range, 
                     potentiel_categories=None, magnitude_categories=None, profondeur_categories=None):
    """Applique les filtres sélectionnés au dataframe"""
    df_filtered = df.copy()
    
    # Filtrer par année
    df_filtered = df_filtered[(df_filtered['Annee'] >= annee_range[0]) & 
                            (df_filtered['Annee'] <= annee_range[1])]
    
    # Filtrer par mois
    df_filtered = df_filtered[df_filtered['Mois'].isin(mois_selected)]
    
    # Filtrer par magnitude
    df_filtered = df_filtered[(df_filtered['Magnitude'] >= magnitude_range[0]) & 
                            (df_filtered['Magnitude'] <= magnitude_range[1])]
    
    # Filtrer par profondeur
    df_filtered = df_filtered[(df_filtered['Profondeur'] >= profondeur_range[0]) & 
                            (df_filtered['Profondeur'] <= profondeur_range[1])]
    
    # Filtrer par catégorie de potentiel destructeur
    if potentiel_categories:
        df_filtered = df_filtered[df_filtered['Potentiel_Categorie'].isin(potentiel_categories)]
    
    # Filtrer par catégorie de magnitude
    if magnitude_categories:
        df_filtered = df_filtered[df_filtered['Magnitude_Categorie'].isin(magnitude_categories)]
    
    # Filtrer par catégorie de profondeur
    if profondeur_categories:
        df_filtered = df_filtered[df_filtered['Profondeur_Categorie'].isin(profondeur_categories)]
    
    return df_filtered

# 1. Analyse de la distribution des magnitudes
def analyser_distribution_magnitudes(df_filtered):
    """Analyse la distribution des magnitudes des séismes"""
    
    # 1.1 Distribution globale des magnitudes
    plt.figure(figsize=(14, 6))
    sns.histplot(df_filtered['Magnitude'], bins=30, kde=True)
    plt.title('Distribution des magnitudes des séismes')
    plt.xlabel('Magnitude')
    plt.ylabel('Nombre de séismes')
    plt.grid(alpha=0.3)
    plt.axvline(x=df_filtered['Magnitude'].mean(), color='r', linestyle='--', 
               label=f'Moyenne: {df_filtered["Magnitude"].mean():.2f}')
    plt.axvline(x=df_filtered['Magnitude'].median(), color='g', linestyle='--', 
               label=f'Médiane: {df_filtered["Magnitude"].median():.2f}')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # 1.2 Distribution par catégorie de magnitude
    plt.figure(figsize=(14, 6))
    # Obtenir les catégories dans l'ordre croissant de magnitude
    order = ['Micro', 'Faible', 'Léger', 'Modéré', 'Fort', 'Majeur', 'Grand']
    # Filtrer pour n'inclure que les catégories présentes dans les données
    order = [cat for cat in order if cat in df_filtered['Magnitude_Categorie'].unique()]
    
    mag_counts = df_filtered['Magnitude_Categorie'].value_counts().reindex(order)
    
    # Créer un dégradé de couleurs bleu à rouge
    palette = sns.color_palette("YlOrRd", len(order))
    
    ax = sns.barplot(x=mag_counts.index, y=mag_counts.values, palette=palette)
    plt.title('Nombre de séismes par catégorie de magnitude')
    plt.xlabel('Catégorie de magnitude')
    plt.ylabel('Nombre de séismes')
    
    # Ajouter les pourcentages sur chaque barre
    total = len(df_filtered)
    for i, count in enumerate(mag_counts.values):
        percentage = count / total * 100
        ax.text(i, count + 5, f'{percentage:.1f}%', ha='center')
    
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Afficher les statistiques de magnitude
    print("\nStatistiques des magnitudes:")
    print(f"Nombre total de séismes: {len(df_filtered)}")
    print(f"Magnitude minimale: {df_filtered['Magnitude'].min():.2f}")
    print(f"Magnitude maximale: {df_filtered['Magnitude'].max():.2f}")
    print(f"Magnitude moyenne: {df_filtered['Magnitude'].mean():.2f}")
    print(f"Magnitude médiane: {df_filtered['Magnitude'].median():.2f}")
    print(f"Écart-type des magnitudes: {df_filtered['Magnitude'].std():.2f}")
    
    print("\nRépartition par catégorie de magnitude:")
    percentage_by_category = df_filtered['Magnitude_Categorie'].value_counts(normalize=True) * 100
    counts_by_category = df_filtered['Magnitude_Categorie'].value_counts()
    
    for category in order:
        if category in percentage_by_category:
            print(f"{category}: {counts_by_category[category]} séismes ({percentage_by_category[category]:.1f}%)")

# 2. Analyse de la distribution des profondeurs
def analyser_distribution_profondeurs(df_filtered):
    """Analyse la distribution des profondeurs des séismes"""
    
    # 2.1 Distribution globale des profondeurs
    plt.figure(figsize=(14, 6))
    sns.histplot(df_filtered['Profondeur'], bins=30, kde=True)
    plt.title('Distribution des profondeurs des séismes')
    plt.xlabel('Profondeur (km)')
    plt.ylabel('Nombre de séismes')
    plt.grid(alpha=0.3)
    plt.axvline(x=df_filtered['Profondeur'].mean(), color='r', linestyle='--', 
               label=f'Moyenne: {df_filtered["Profondeur"].mean():.2f} km')
    plt.axvline(x=df_filtered['Profondeur'].median(), color='g', linestyle='--', 
               label=f'Médiane: {df_filtered["Profondeur"].median():.2f} km')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # 2.2 Distribution par catégorie de profondeur
    plt.figure(figsize=(14, 6))
    # Obtenir les catégories dans l'ordre croissant de profondeur
    order = ['Peu profond', 'Intermédiaire', 'Profond']
    # Filtrer pour n'inclure que les catégories présentes dans les données
    order = [cat for cat in order if cat in df_filtered['Profondeur_Categorie'].unique()]
    
    depth_counts = df_filtered['Profondeur_Categorie'].value_counts().reindex(order)
    
    # Créer un dégradé de couleurs bleu
    palette = sns.color_palette("Blues", len(order))
    
    ax = sns.barplot(x=depth_counts.index, y=depth_counts.values, palette=palette)
    plt.title('Nombre de séismes par catégorie de profondeur')
    plt.xlabel('Catégorie de profondeur')
    plt.ylabel('Nombre de séismes')
    
    # Ajouter les pourcentages sur chaque barre
    total = len(df_filtered)
    for i, count in enumerate(depth_counts.values):
        percentage = count / total * 100
        ax.text(i, count + 5, f'{percentage:.1f}%', ha='center')
    
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Afficher les statistiques de profondeur
    print("\nStatistiques des profondeurs:")
    print(f"Profondeur minimale: {df_filtered['Profondeur'].min():.2f} km")
    print(f"Profondeur maximale: {df_filtered['Profondeur'].max():.2f} km")
    print(f"Profondeur moyenne: {df_filtered['Profondeur'].mean():.2f} km")
    print(f"Profondeur médiane: {df_filtered['Profondeur'].median():.2f} km")
    print(f"Écart-type des profondeurs: {df_filtered['Profondeur'].std():.2f} km")
    
    print("\nRépartition par catégorie de profondeur:")
    percentage_by_category = df_filtered['Profondeur_Categorie'].value_counts(normalize=True) * 100
    counts_by_category = df_filtered['Profondeur_Categorie'].value_counts()
    
    for category in order:
        if category in percentage_by_category:
            print(f"{category}: {counts_by_category[category]} séismes ({percentage_by_category[category]:.1f}%)")

# 3. Analyse de la relation entre magnitude et profondeur
def analyser_relation_magnitude_profondeur(df_filtered):
    """Analyse la relation entre magnitude et profondeur des séismes"""
    
    # 3.1 Nuage de points magnitude vs profondeur
    plt.figure(figsize=(14, 8))
    
    # Utiliser une palette de couleurs pour le potentiel destructeur
    scatter = plt.scatter(df_filtered['Profondeur'], df_filtered['Magnitude'], 
                         c=df_filtered['Potentiel_Destructeur'], cmap='YlOrRd', 
                         alpha=0.7, s=50)
    
    plt.colorbar(scatter, label='Potentiel destructeur')
    plt.title('Relation entre magnitude et profondeur des séismes')
    plt.xlabel('Profondeur (km)')
    plt.ylabel('Magnitude')
    
    # Ajouter une courbe de régression
    if len(df_filtered) > 2:
        try:
            # Régression linéaire simple
            slope, intercept, r_value, p_value, std_err = stats.linregress(
                df_filtered['Profondeur'], df_filtered['Magnitude'])
            
            x = np.array([df_filtered['Profondeur'].min(), df_filtered['Profondeur'].max()])
            y = intercept + slope * x
            
            plt.plot(x, y, 'b--', 
                   label=f'Régression: y={slope:.4f}x+{intercept:.4f}, r²={r_value**2:.3f}')
            plt.legend()
            
            print(f"\nAnalyse de corrélation entre magnitude et profondeur:")
            print(f"Coefficient de corrélation (r): {r_value:.3f}")
            print(f"Coefficient de détermination (r²): {r_value**2:.3f}")
            print(f"p-value: {p_value:.4f}")
            
            if p_value < 0.05:
                print("Il existe une relation statistiquement significative entre la magnitude et la profondeur.")
                if slope > 0:
                    print("La magnitude tend à augmenter avec la profondeur.")
                else:
                    print("La magnitude tend à diminuer avec la profondeur.")
            else:
                print("Aucune relation statistiquement significative n'a été détectée entre la magnitude et la profondeur.")
        except:
            print("Impossible de calculer la régression avec les données actuelles.")
    
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 3.2 Heatmap magnitude vs profondeur
    plt.figure(figsize=(14, 8))
    
    # Créer des bins pour la magnitude et la profondeur
    mag_bins = np.linspace(df_filtered['Magnitude'].min(), df_filtered['Magnitude'].max(), 15)
    depth_bins = np.linspace(df_filtered['Profondeur'].min(), df_filtered['Profondeur'].max(), 15)
    
    # Créer un histogramme 2D
    heatmap, xedges, yedges = np.histogram2d(df_filtered['Profondeur'], df_filtered['Magnitude'], 
                                           bins=[depth_bins, mag_bins])
    
    # Afficher la heatmap
    plt.pcolormesh(xedges, yedges, heatmap.T, cmap='YlOrRd', shading='auto')
    plt.colorbar(label='Nombre de séismes')
    plt.title('Heatmap de la relation entre magnitude et profondeur')
    plt.xlabel('Profondeur (km)')
    plt.ylabel('Magnitude')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 3.3 Box plots de magnitude par catégorie de profondeur
    plt.figure(figsize=(12, 6))
    order = ['Peu profond', 'Intermédiaire', 'Profond']
    order = [cat for cat in order if cat in df_filtered['Profondeur_Categorie'].unique()]
    
    sns.boxplot(x='Profondeur_Categorie', y='Magnitude', data=df_filtered, order=order, palette='Blues')
    plt.title('Distribution des magnitudes par catégorie de profondeur')
    plt.xlabel('Catégorie de profondeur')
    plt.ylabel('Magnitude')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Statistiques par catégorie de profondeur
    print("\nStatistiques de magnitude par catégorie de profondeur:")
    for category in order:
        if category in df_filtered['Profondeur_Categorie'].unique():
            mag_stats = df_filtered[df_filtered['Profondeur_Categorie'] == category]['Magnitude'].describe()
            print(f"\n{category}:")
            print(f"  Nombre de séismes: {mag_stats['count']:.0f}")
            print(f"  Magnitude moyenne: {mag_stats['mean']:.2f}")
            print(f"  Magnitude médiane: {mag_stats['50%']:.2f}")
            print(f"  Magnitude maximale: {mag_stats['max']:.2f}")
            print(f"  Écart-type: {mag_stats['std']:.2f}")

# 4. Analyse du potentiel destructeur
def analyser_potentiel_destructeur(df_filtered):
    """Analyse le potentiel destructeur des séismes (combinaison de magnitude et profondeur)"""
    
    # 4.1 Distribution du potentiel destructeur
    plt.figure(figsize=(14, 6))
    sns.histplot(df_filtered['Potentiel_Destructeur'], bins=30, kde=True)
    plt.title('Distribution du potentiel destructeur des séismes')
    plt.xlabel('Potentiel destructeur')
    plt.ylabel('Nombre de séismes')
    plt.grid(alpha=0.3)
    plt.axvline(x=df_filtered['Potentiel_Destructeur'].mean(), color='r', linestyle='--', 
               label=f'Moyenne: {df_filtered["Potentiel_Destructeur"].mean():.2f}')
    plt.axvline(x=df_filtered['Potentiel_Destructeur'].median(), color='g', linestyle='--', 
               label=f'Médiane: {df_filtered["Potentiel_Destructeur"].median():.2f}')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # 4.2 Distribution par catégorie de potentiel destructeur
    plt.figure(figsize=(14, 6))
    # Obtenir les catégories dans l'ordre croissant de potentiel
    order = ['Très faible', 'Faible', 'Modéré', 'Élevé', 'Très élevé']
    # Filtrer pour n'inclure que les catégories présentes dans les données
    order = [cat for cat in order if cat in df_filtered['Potentiel_Categorie'].unique()]
    
    potentiel_counts = df_filtered['Potentiel_Categorie'].value_counts().reindex(order)
    
    # Créer un dégradé de couleurs
    palette = sns.color_palette("YlOrRd", len(order))
    
    ax = sns.barplot(x=potentiel_counts.index, y=potentiel_counts.values, palette=palette)
    plt.title('Nombre de séismes par catégorie de potentiel destructeur')
    plt.xlabel('Catégorie de potentiel destructeur')
    plt.ylabel('Nombre de séismes')
    
    # Ajouter les pourcentages sur chaque barre
    total = len(df_filtered)
    for i, count in enumerate(potentiel_counts.values):
        percentage = count / total * 100
        ax.text(i, count + 5, f'{percentage:.1f}%', ha='center')
    
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 4.3 Carte de chaleur du potentiel destructeur (magnitude vs profondeur)
    plt.figure(figsize=(12, 8))
    
    # Créer une grille pour la carte de chaleur
    x = np.linspace(df_filtered['Profondeur'].min(), df_filtered['Profondeur'].max(), 100)
    y = np.linspace(df_filtered['Magnitude'].min(), df_filtered['Magnitude'].max(), 100)
    X, Y = np.meshgrid(x, y)
    Z = Y * (1 + 70/X)  # Formule du potentiel destructeur
    
    # Tracer la carte de chaleur
    contour = plt.contourf(X, Y, Z, 20, cmap='YlOrRd')
    plt.colorbar(contour, label='Potentiel destructeur')
    
    # Superposer les points des séismes réels
    plt.scatter(df_filtered['Profondeur'], df_filtered['Magnitude'], 
               c='black', alpha=0.5, s=20)
    
    plt.title('Carte du potentiel destructeur (magnitude vs profondeur)')
    plt.xlabel('Profondeur (km)')
    plt.ylabel('Magnitude')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Statistiques du potentiel destructeur
    print("\nStatistiques du potentiel destructeur:")
    print(f"Potentiel destructeur minimum: {df_filtered['Potentiel_Destructeur'].min():.2f}")
    print(f"Potentiel destructeur maximum: {df_filtered['Potentiel_Destructeur'].max():.2f}")
    print(f"Potentiel destructeur moyen: {df_filtered['Potentiel_Destructeur'].mean():.2f}")
    print(f"Potentiel destructeur médian: {df_filtered['Potentiel_Destructeur'].median():.2f}")
    print(f"Écart-type du potentiel destructeur: {df_filtered['Potentiel_Destructeur'].std():.2f}")
    
    print("\nRépartition par catégorie de potentiel destructeur:")
    percentage_by_category = df_filtered['Potentiel_Categorie'].value_counts(normalize=True) * 100
    counts_by_category = df_filtered['Potentiel_Categorie'].value_counts()
    
    for category in order:
        if category in percentage_by_category:
            print(f"{category}: {counts_by_category[category]} séismes ({percentage_by_category[category]:.1f}%)")

# 5. Analyse de l'énergie libérée
def analyser_energie(df_filtered):
    """Analyse l'énergie libérée par les séismes"""
    
    # 5.1 Distribution de l'énergie (échelle logarithmique)
    plt.figure(figsize=(14, 6))
    sns.histplot(np.log10(df_filtered['Energie']), bins=30, kde=True)
    plt.title('Distribution de l\'énergie libérée par les séismes (échelle logarithmique)')
    plt.xlabel('Énergie (log10 Joules)')
    plt.ylabel('Nombre de séismes')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 5.2 Énergie cumulée dans le temps
    plt.figure(figsize=(14, 6))
    
    # Trier les données par date
    df_sorted = df_filtered.sort_values('Date')
    
    # Calculer l'énergie cumulée
    energie_cumulee = df_sorted['Energie'].cumsum()
    
    plt.plot(df_sorted['Date'], energie_cumulee)
    plt.title('Énergie sismique cumulée au fil du temps')
    plt.xlabel('Date')
    plt.ylabel('Énergie cumulée (Joules)')
    plt.yscale('log')  # Échelle logarithmique pour mieux visualiser
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 5.3 Comparaison de l'énergie par catégorie de magnitude
    plt.figure(figsize=(14, 6))
    
    # Obtenir les catégories dans l'ordre croissant
    order = ['Micro', 'Faible', 'Léger', 'Modéré', 'Fort', 'Majeur', 'Grand']
    order = [cat for cat in order if cat in df_filtered['Magnitude_Categorie'].unique()]
    
    # Calculer l'énergie totale par catégorie
    energie_par_categorie = df_filtered.groupby('Magnitude_Categorie')['Energie'].sum()
    energie_par_categorie = energie_par_categorie.reindex(order)
    
    # Création d'un dégradé de couleurs
    palette = sns.color_palette("YlOrRd", len(order))
    
    # Graphique en échelle logarithmique
    plt.figure(figsize=(14, 6))
    ax = sns.barplot(x=energie_par_categorie.index, y=energie_par_categorie.values, palette=palette)
    plt.title('Énergie totale libérée par catégorie de magnitude')
    plt.xlabel('Catégorie de magnitude')
    plt.ylabel('Énergie totale (Joules)')
    plt.yscale('log')
    plt.grid(alpha=0.3)
    
    # Ajouter les pourcentages sur chaque barre
    total = energie_par_categorie.sum()
    for i, energy in enumerate(energie_par_categorie.values):
        percentage = energy / total * 100
        ax.text(i, energy * 1.1, f'{percentage:.1f}%', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques d'énergie
    print("\nStatistiques de l'énergie libérée:")
    print(f"Énergie totale libérée: {df_filtered['Energie'].sum():.2e} Joules")
    print(f"Énergie moyenne par séisme: {df_filtered['Energie'].mean():.2e} Joules")
    print(f"Énergie médiane par séisme: {df_filtered['Energie'].median():.2e} Joules")
    print(f"Séisme le plus énergétique: {df_filtered['Energie'].max():.2e} Joules")
    
    print("\nRépartition de l'énergie par catégorie de magnitude:")
    total_energy = df_filtered['Energie'].sum()
    
    for category in order:
        if category in df_filtered['Magnitude_Categorie'].unique():
            category_energy = df_filtered[df_filtered['Magnitude_Categorie'] == category]['Energie'].sum()
            percentage = category_energy / total_energy * 100
            count = len(df_filtered[df_filtered['Magnitude_Categorie'] == category])
            print(f"{category}: {category_energy:.2e} Joules ({percentage:.1f}% de l'énergie totale, {count} séismes)")

# Création du Dashboard
# Correction pour la fonction creer_dashboard
def creer_dashboard():
    # Charger les données
    df = charger_donnees()
    
    # Créer les widgets pour les filtres
    output_dashboard = widgets.Output()
    
    # Filtres temporels
    annees = sorted(df['Annee'].unique())
    mois = list(range(1, 13))
    mois_noms = ['Janvier', 'Février', 'Mars', 'Avril', 'Mai', 'Juin', 
                'Juillet', 'Août', 'Septembre', 'Octobre', 'Novembre', 'Décembre']
    mois_dict = {i+1: nom for i, nom in enumerate(mois_noms)}
    
    # Filtres magnitude et profondeur
    min_mag = float(df['Magnitude'].min())
    max_mag = float(df['Magnitude'].max())
    min_prof = float(df['Profondeur'].min())
    max_prof = float(df['Profondeur'].max())
    
    # Filtres par catégories
    mag_categories = sorted(df['Magnitude_Categorie'].unique())
    prof_categories = sorted(df['Profondeur_Categorie'].unique())
    pot_categories = sorted(df['Potentiel_Categorie'].unique())
    
    # Création des widgets
    annee_slider = widgets.IntRangeSlider(
        value=[annees[0], annees[-1]],
        min=annees[0],
        max=annees[-1],
        step=1,
        description='Années:',
        continuous_update=False,
        layout=widgets.Layout(width='70%')
    )
    
    mois_checkbox = widgets.SelectMultiple(
        options=[(mois_dict[m], m) for m in mois],
        value=mois,
        description='Mois:',
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    magnitude_slider = widgets.FloatRangeSlider(
        value=[min_mag, max_mag],
        min=min_mag,
        max=max_mag,
        step=0.1,
        description='Magnitude:',
        continuous_update=False,
        layout=widgets.Layout(width='70%')
    )
    
    profondeur_slider = widgets.FloatRangeSlider(
        value=[min_prof, max_prof],
        min=min_prof,
        max=max_prof,
        step=5,
        description='Profondeur:',
        continuous_update=False,
        layout=widgets.Layout(width='70%')
    )
    
    # Sélection multiple pour les catégories
    magnitude_cat_select = widgets.SelectMultiple(
        options=mag_categories,
        value=mag_categories,
        description='Cat. Magnitude:',
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    profondeur_cat_select = widgets.SelectMultiple(
        options=prof_categories,
        value=prof_categories,
        description='Cat. Profondeur:',
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    potentiel_cat_select = widgets.SelectMultiple(
        options=pot_categories,
        value=pot_categories,
        description='Cat. Potentiel:',
        layout=widgets.Layout(width='50%', height='100px')
    )
    
    # Types d'analyse
    analyse_type = widgets.RadioButtons(
        options=['Distribution des magnitudes', 'Distribution des profondeurs', 
                'Relation magnitude/profondeur', 'Potentiel destructeur', 
                'Énergie libérée', 'Analyse par clustering'],  # Ajout de l'option clustering
        description='Type d\'analyse:',
        layout=widgets.Layout(width='50%')
    )
    
    filtrer_button = widgets.Button(
        description='Analyser caractéristiques',
        button_style='primary',
        tooltip='Cliquez pour analyser les caractéristiques',
        layout=widgets.Layout(width='200px')
    )
    
    # Fonction principale pour mettre à jour l'analyse
    def update_dashboard(b):
        with output_dashboard:
            clear_output(wait=True)
            
            # Récupérer les valeurs des filtres
            annee_range = annee_slider.value
            mois_selected = mois_checkbox.value
            magnitude_range = magnitude_slider.value
            profondeur_range = profondeur_slider.value
            
            # Récupérer les catégories sélectionnées
            mag_cats_selected = magnitude_cat_select.value
            prof_cats_selected = profondeur_cat_select.value
            pot_cats_selected = potentiel_cat_select.value
            
            # Appliquer les filtres
            df_filtered = appliquer_filtres(df, annee_range, mois_selected, magnitude_range, profondeur_range,
                                         pot_cats_selected, mag_cats_selected, prof_cats_selected)
            
            print(f"Données filtrées: {len(df_filtered)} séismes sur {len(df)} ({len(df_filtered)/len(df)*100:.1f}%)")
            
            if len(df_filtered) == 0:
                print("Aucune donnée ne correspond aux critères de filtrage!")
                return
            
            # Effectuer l'analyse sélectionnée
            analysis_type = analyse_type.value
            
            if analysis_type == 'Distribution des magnitudes':
                analyser_distribution_magnitudes(df_filtered)
                
            elif analysis_type == 'Distribution des profondeurs':
                analyser_distribution_profondeurs(df_filtered)
                
            elif analysis_type == 'Relation magnitude/profondeur':
                analyser_relation_magnitude_profondeur(df_filtered)
                
            elif analysis_type == 'Potentiel destructeur':
                analyser_potentiel_destructeur(df_filtered)
                
            elif analysis_type == 'Énergie libérée':
                analyser_energie(df_filtered)
                
            elif analysis_type == 'Analyse par clustering':
                analyser_clustering(df_filtered)
    
    # Connecter le bouton à la fonction d'actualisation
    filtrer_button.on_click(update_dashboard)
    
    # Afficher l'interface
    print("\n--- DASHBOARD D'ANALYSE DES CARACTÉRISTIQUES SISMIQUES ---")
    
    # Organisation des filtres en accordéon
    accordion = widgets.Accordion(
        children=[
            widgets.VBox([
                widgets.HBox([annee_slider]),
                widgets.HBox([mois_checkbox])
            ]),
            widgets.VBox([
                widgets.HBox([magnitude_slider]),
                widgets.HBox([profondeur_slider])
            ]),
            widgets.VBox([
                widgets.HBox([magnitude_cat_select, profondeur_cat_select, potentiel_cat_select])
            ])
        ]
    )
    accordion.set_title(0, 'Filtres temporels')
    accordion.set_title(1, 'Filtres numériques')
    accordion.set_title(2, 'Filtres catégoriels')
    
    controls = widgets.VBox([
        accordion,
        widgets.HBox([widgets.Label('Type d\'analyse:', style={'font_weight': 'bold'})]),
        widgets.HBox([analyse_type]),
        widgets.HBox([filtrer_button])
    ])
    
    display(controls)
    display(output_dashboard)
    
    # Lancer l'analyse avec les filtres par défaut
    update_dashboard(None)

# Lancer le dashboard
if __name__ == "__main__":
    creer_dashboard()

Tentative de chargement du fichier: NewDataseisme_corriger.csv
Fichier non trouvé: NewDataseisme_corriger.csv
Tentative de chargement du fichier: NewDataseisme_corrige.csv
Fichier chargé avec succès: NewDataseisme_corrige.csv

Aperçu des données:
               Date    Magnitude    Latitude  Longitude Profondeur  origine
0  08/05/2025 20:34  1,998769442  -12,750200  45,561000      51,54        5
1  08/05/2025 20:27  2,374112553  -12,788200  45,619300      47,24        5
2  08/05/2025 19:16  1,667391108  -12,566000  45,175300      38,39        5
3  08/05/2025 01:04  2,025011775  -12,805300  45,580500      49,82        5
4  07/05/2025 18:46  1,282582543  -12,805500  45,347200      43,46        5

Exemples de format de date dans les données:
08/05/2025 20:34
08/05/2025 20:27
08/05/2025 19:16
08/05/2025 01:04
07/05/2025 18:46

Conversion des dates avec parse_date_flexible...
Conversion des dates: 14216 succès, 0 échecs

Attention: 6 valeurs de profondeur négatives détectées.
Application de

Output()